## Ideal Simulation

In [ ]:
# Install (uncomment if you need to install or upgrade in notebook)
# !pip install qiskit qiskit-aer qiskit-optimization qiskit_algorithms

import time
import itertools
import math
import sys
import numpy as np

# Qiskit optimization primitives
from qiskit_optimization import QuadraticProgram
from qiskit_optimization.converters import QuadraticProgramToQubo
from qiskit_optimization.algorithms import MinimumEigenOptimizer
# QAOA from qiskit_optimization
from qiskit_optimization.minimum_eigensolvers import QAOA

# Try SamplerV2 (aer) and AerSimulator with multiple constructor signatures (robust)
SamplerV2 = None
try:
    from qiskit_aer.primitives import SamplerV2 as _SamplerV2
    SamplerV2 = _SamplerV2
except Exception:
    try:
        # alternate packaging
        from qiskit.primitives import Sampler as _SamplerV2
        SamplerV2 = _SamplerV2
    except Exception:
        SamplerV2 = None

from qiskit_aer import AerSimulator  

# Try optimizers: SPSA preferred for noisy quantum, else COBYLA, else SciPy wrapper
optimizer = None
try:
    from qiskit.algorithms.optimizers import SPSA
    # keep low iterations for quick runs; tune later
    optimizer = SPSA(maxiter=50)
    print("Using SPSA optimizer from qiskit.algorithms.optimizers")
except Exception:
    try:
        from qiskit.algorithms.optimizers import COBYLA
        optimizer = COBYLA(maxiter=200)
        print("Using COBYLA optimizer from qiskit.algorithms.optimizers")
    except Exception:
        # SciPy fallback wrapper with minimize interface (for QAOA constructor compatibility)
        from scipy.optimize import minimize as scipy_minimize
        class SciPyOptimizerWrapper:
            def __init__(self, method='COBYLA', options=None):
                self.method = method
                self.options = options or {"maxiter":200}
            def optimize(self, num_vars, objective, initial_point=None):
                def obj_for_scipy(x):
                    val = objective(x)
                    return float(val) if not isinstance(val, (tuple,list)) else float(val[0])
                x0 = initial_point if initial_point is not None else [0.0]*num_vars
                res = scipy_minimize(obj_for_scipy, x0, method=self.method, options=self.options)
                return res.x, float(res.fun), getattr(res, "nfev", None)
            def minimize(self, fun, x0, **kwargs):
                method = kwargs.pop("method", self.method)
                options = kwargs.pop("options", self.options)
                res = scipy_minimize(fun, x0, method=method, options=options, **kwargs)
                return res
        optimizer = SciPyOptimizerWrapper(method='COBYLA', options={"maxiter":200})
        print("Using SciPy-based optimizer wrapper")

# Prepare a robust SamplerV2/AerSimulator instance (tries multiple constructor signatures)
sampler = None
sampler_errors = []
constructors = []
if SamplerV2 is not None:
    constructors = [
        lambda: SamplerV2(mode=AerSimulator()),
        lambda: SamplerV2(backend=AerSimulator()),
        lambda: SamplerV2(),
    ]
else:
    constructors = [lambda: AerSimulator()]

for ctor in constructors:
    try:
        sampler = ctor()
        print("Sampler created with constructor:", ctor)
        break
    except Exception as e:
        sampler_errors.append(e)

if sampler is None:
    print("Sampler constructors tried and failed. Errors:")
    for i, e in enumerate(sampler_errors, 1):
        print(f"[{i}] {type(e).__name__}: {e}")
    raise RuntimeError("Unable to instantiate Sampler/AerSimulator. Ensure qiskit-aer and primitives are installed.")

# ---- Problem data ----
distance_matrix = np.array(
      [[ 0.   ,  8.628, 11.496,  9.445, 10.852,  9.672], # H
       [14.194,  0.   ,  2.361, 10.922,  9.238,  9.43 ], # P1 (DT)
       [17.785,  7.745,  0.   , 11.808, 10.124, 10.478], # P2 (GR)
       [11.864, 19.672, 15.661,  0.   , 11.572, 11.538], # P3 (R2)
       [ 7.343, 12.172, 10.053,  4.071,  0.   ,  5.931], # P4 (R3_2)
       [ 9.269,  9.398, 12.266,  7.318,  8.725,  0.   ]]  # P5 (IT)
)
locations = ['H', 'DT', 'GR', 'R2', 'R3_2', 'IT']
patient_indices = [1,2,3,4,5]
penalty = 1000

# Helper: build TSP QuadraticProgram for a local subproblem (nodes = hospital + chosen patients)
def build_tsp_quadratic_program(dist_local, penalty=penalty):
    n = dist_local.shape[0]
    qp = QuadraticProgram(name="TSP_local")
    # binary vars x_i_j (i != j)  using local indices 0..n-1
    for i in range(n):
        for j in range(n):
            if i == j: continue
            qp.binary_var(name=f"x_{i}_{j}")
    # linear objective
    linear = {f"x_{i}_{j}": float(dist_local[i,j]) for i in range(n) for j in range(n) if i!=j}
    qp.minimize(linear=linear)
    # constraints: arrive and depart exactly once
    for i in range(n):
        arrive = {f"x_{j}_{i}": 1 for j in range(n) if j!=i}
        qp.linear_constraint(linear=arrive, sense="==", rhs=1, name=f"arrive_{i}")
        depart = {f"x_{i}_{j}": 1 for j in range(n) if j!=i}
        qp.linear_constraint(linear=depart, sense="==", rhs=1, name=f"depart_{i}")
    # convert to QUBO
    qubo = QuadraticProgramToQubo(penalty=penalty).convert(qp)
    return qp, qubo

# Helper: reconstruct tour from result.variables_dict (local indices)
def reconstruct_tour_from_solution(variables_dict):
    path_dict = {}
    for varname, val in variables_dict.items():
        try:
            if float(val) > 0.5:
                parts = varname.split('_')
                # expecting "x_i_j"
                if len(parts) >= 3:
                    si = int(parts[1]); sj = int(parts[2])
                    path_dict[si] = sj
        except Exception:
            continue
    # walk from 0
    path = [0]
    cur = 0
    while True:
        nxt = path_dict.get(cur, None)
        if nxt is None: break
        if nxt in path:
            # avoid loops
            break
        path.append(nxt)
        cur = nxt
        if len(path) > 50: break
    # optionally append return to start
    if len(path) >= 1 and path[-1] != 0 and len(path) == len(path_dict)+1:
        path.append(0)
    return path

# Top-level hybrid search: enumerate all combinations of 3 patients for Trip A
best = {
    "distance": float('inf'),
    "trip_A_group": None,
    "trip_B_group": None,
    "trip_A_route": None,
    "trip_B_route": None,
    "trip_A_dist": None,
    "trip_B_dist": None,
    "qaoa_params": None
}

print("Starting hybrid search: Trip A quantum (QAOA) on 3 patients, Trip B classical (2 patients).")
for idx, group_A in enumerate(itertools.combinations(patient_indices, 3), start=1):
    print(f"\n--- Combination {idx}/10: Trip A patients = {[locations[i] for i in group_A]} ---")
    group_B = tuple(p for p in patient_indices if p not in group_A)

    # Build local distance matrix for Trip A (hospital + the 3 chosen patients)
    trip_A_nodes = (0,) + group_A  # local mapping -> global indices
    dist_A_local = distance_matrix[np.ix_(trip_A_nodes, trip_A_nodes)]
    n_local = dist_A_local.shape[0]
    print(f"Local problem size n_local={n_local} nodes -> expected qubits m = n_local*(n_local-1) = {n_local*(n_local-1)}")

    # Build quadratic program and QUBO
    qp_local, qubo_local = build_tsp_quadratic_program(dist_A_local, penalty=penalty)

    # Instantiate QAOA with sampler + optimizer (try some constructor orders like in your working code)
    qaoa = None
    qaoa_errors = []
    attempts = [
        {"sampler": sampler, "optimizer": optimizer, "reps": 2},
        {"optimizer": optimizer, "sampler": sampler, "reps": 2},
        {"sampler": sampler, "reps": 2, "optimizer": optimizer},
    ]
    for kwargs in attempts:
        try:
            qaoa = QAOA(**kwargs)
            print("QAOA instantiated with args:", {k:v for k,v in kwargs.items() if k!='sampler'})
            break
        except Exception as e:
            qaoa_errors.append((kwargs, e))

    if qaoa is None:
        print("QAOA instantiation attempts failed. Details:")
        for kw, err in qaoa_errors:
            print("Attempt:", {k: (v if k!='sampler' else "<sampler>") for k,v in kw.items()})
            print(" ->", type(err).__name__, err)
        raise RuntimeError("QAOA could not be instantiated in this environment. No fallback (quantum required).")

    # Wrap in MinimumEigenOptimizer (try both calling conventions)
    try:
        meo = MinimumEigenOptimizer(min_eigen_solver=qaoa)
    except TypeError:
        meo = MinimumEigenOptimizer(qaoa)

    # Solve using quantum QAOA
    print("Running MinimumEigenOptimizer (QAOA) on QUBO (this will call the sampler)...")
    t0 = time.time()
    result = meo.solve(qubo_local)
    t1 = time.time()
    print(f"QAOA solve finished in {t1-t0:.3f} s. Status: {result.status if hasattr(result,'status') else 'unknown'}")
    print(f"Raw QUBO objective (result.fval): {result.fval:.6f}")

    # Reconstruct local tour and convert to global node labels
    path_local = reconstruct_tour_from_solution(result.variables_dict)
    # Ensure path_local includes all nodes; if not, still convert what we have but warn
    if len(path_local) < n_local:
        print("Warning: extracted path is incomplete (may be due to QAOA approximation). Extracted local path:", path_local)
    path_global = [trip_A_nodes[i] for i in path_local]

    # Compute trip A distance (use brute force to compute true route cost of chosen permutation)
    # If the solution contains full ordering (excluding final return 0), evaluate it; otherwise recompute best permutation among selected nodes
    tripA_dist = None
    if len(path_local) == n_local and path_local[0] == 0:
        # full order found
        # ensure return to 0 at end
        tour = path_local + ([0] if path_local[-1] != 0 else [])
        # compute cost
        tripA_dist = 0.0
        for a,b in zip(tour[:-1], tour[1:]):
            tripA_dist += float(dist_A_local[a,b])
    else:
        # fallback: interpret variables' selected edges to derive the set of visited patients and brute-force the best order among them
        # Build set of selected patient local indices (exclude 0)
        selected = sorted([j for j in range(1,n_local) if any((f"x_{i}_{j}" in result.variables_dict and result.variables_dict[f"x_{i}_{j}"]>0.5) for i in range(n_local) if i!=j)])
        if not selected:
            raise RuntimeError("QAOA returned no selected patient edges; aborting.")
        # brute-force best order among selected patients
        best_perm = None; best_perm_cost = math.inf
        for perm in itertools.permutations(selected):
            tour = [0] + list(perm) + [0]
            cost = sum(float(dist_A_local[a,b]) for a,b in zip(tour[:-1], tour[1:]))
            if cost < best_perm_cost:
                best_perm_cost = cost; best_perm = perm
        tripA_dist = best_perm_cost
        # set path_global accordingly
        path_global = [trip_A_nodes[i] for i in ([0] + list(best_perm) + [0])]
        print("QAOA didn't give a clean Hamiltonian cycle; selected patients:", [trip_A_nodes[i] for i in selected])
        print("Using brute-force best permutation among selected patients:", best_perm, "cost:", best_perm_cost)

    # --- Trip B (2 patients) solved classically exactly ---
    p1, p2 = group_B
    # two possible routes: H->p1->p2->H or H->p2->p1->H (global indexing)
    dist_B_option1 = distance_matrix[0,p1] + distance_matrix[p1,p2] + distance_matrix[p2,0]
    dist_B_option2 = distance_matrix[0,p2] + distance_matrix[p2,p1] + distance_matrix[p1,0]
    if dist_B_option1 <= dist_B_option2:
        tripB_dist = float(dist_B_option1)
        pathB_global = [0, p1, p2, 0]
    else:
        tripB_dist = float(dist_B_option2)
        pathB_global = [0, p2, p1, 0]

    total = float(tripA_dist) + float(tripB_dist)
    print(f"Trip A distance (evaluated): {tripA_dist:.6f}, Trip B distance: {tripB_dist:.6f}, Total: {total:.6f}")

    # Update best if improved
    if total < best["distance"]:
        best.update({
            "distance": total,
            "trip_A_group": [locations[i] for i in group_A],
            "trip_B_group": [locations[i] for i in group_B],
            "trip_A_route": [locations[i] for i in path_global],
            "trip_B_route": [locations[i] for i in pathB_global],
            "trip_A_dist": tripA_dist,
            "trip_B_dist": tripB_dist,
            "qaoa_time": t1-t0,
            "qaoa_sampler_constructor": type(sampler).__name__
        })
        print(">> New best solution recorded.")

# Final printout
print("\n" + "="*50)
print("FINAL BEST SOLUTION (quantum Trip A):")
print("Total optimal distance (hybrid): {:.6f}".format(best["distance"]))
print("Trip A (3 patients):", best["trip_A_group"], "Route:", " -> ".join(best["trip_A_route"]), f"Distance: {best['trip_A_dist']:.6f}")
print("Trip B (2 patients):", best["trip_B_group"], "Route:", " -> ".join(best["trip_B_route"]), f"Distance: {best['trip_B_dist']:.6f}")
print("QAOA sampler:", best.get("qaoa_sampler_constructor"), "QAOA time (s):", best.get("qaoa_time"))
print("="*50)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 646.5/646.5 kB 9.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 89.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 113.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 237.1/237.1 kB 18.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 327.8/327.8 kB 28.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 93.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 4.7 MB/s eta 0:00:00
  Created wheel for docplex: filename=docplex-2.30.251-py3-none-any.whl size=685954 sha256=767e8284391da35b2965d23fab2e9f1e7a911271eb05d543d3299e9999df96d1
  Stored in directory: /root/.cache/pip/wheels/c1/d9/5c/1d919b1e441ebbaff244513a13d09c3c0378401601d4184bb0
Succ

/usr/local/lib/python3.12/dist-packages/qiskit_optimization/minimum_eigensolvers/qaoa.py:122: UserWarning: Using Sampler V2 (other than StatevectorSampler) without a pass_manager may result in an error. Consider providing a pass_manager for proper circuit transpilation.
  super().__init__(


QAOA solve finished in 2.125 s. Status: OptimizationResultStatus.SUCCESS
Raw QUBO objective (result.fval): 47.045000
Trip A distance (evaluated): 47.045000, Trip B distance: 25.740000, Total: 72.785000
>> New best solution recorded.

--- Combination 2/10: Trip A patients = ['DT', 'GR', 'R3_2'] ---
Local problem size n_local=4 nodes -> expected qubits m = n_local*(n_local-1) = 12
QAOA instantiated with args: {'optimizer': <__main__.SciPyOptimizerWrapper object at 0x7f06c9fd56a0>, 'reps': 2}
Running MinimumEigenOptimizer (QAOA) on QUBO (this will call the sampler)...


/usr/local/lib/python3.12/dist-packages/qiskit_optimization/minimum_eigensolvers/qaoa.py:122: UserWarning: Using Sampler V2 (other than StatevectorSampler) without a pass_manager may result in an error. Consider providing a pass_manager for proper circuit transpilation.
  super().__init__(


QAOA solve finished in 3.437 s. Status: OptimizationResultStatus.SUCCESS
Raw QUBO objective (result.fval): 43.170000
Trip A distance (evaluated): 43.170000, Trip B distance: 28.854000, Total: 72.024000
>> New best solution recorded.

--- Combination 3/10: Trip A patients = ['DT', 'GR', 'IT'] ---
Local problem size n_local=4 nodes -> expected qubits m = n_local*(n_local-1) = 12
QAOA instantiated with args: {'optimizer': <__main__.SciPyOptimizerWrapper object at 0x7f06c9fd56a0>, 'reps': 2}
Running MinimumEigenOptimizer (QAOA) on QUBO (this will call the sampler)...


...

QAOA solve finished in 3.231 s. Status: OptimizationResultStatus.SUCCESS
Raw QUBO objective (result.fval): 48.137000
QAOA didn't give a clean Hamiltonian cycle; selected patients: [2, 3, 5]
Using brute-force best permutation among selected patients: (1, 3, 2) cost: 41.156
Trip A distance (evaluated): 41.156000, Trip B distance: 25.209000, Total: 66.365000

--- Combination 9/10: Trip A patients = ['GR', 'R3_2', 'IT'] ---
Local problem size n_local=4 nodes -> expected qubits m = n_local*(n_local-1) = 12
QAOA instantiated with args: {'optimizer': <__main__.SciPyOptimizerWrapper object at 0x7f06c9fd56a0>, 'reps': 2}
Running MinimumEigenOptimizer (QAOA) on QUBO (this will call the sampler)...


/usr/local/lib/python3.12/dist-packages/qiskit_optimization/minimum_eigensolvers/qaoa.py:122: UserWarning: Using Sampler V2 (other than StatevectorSampler) without a pass_manager may result in an error. Consider providing a pass_manager for proper circuit transpilation.
  super().__init__(


QAOA solve finished in 1.585 s. Status: OptimizationResultStatus.SUCCESS
Raw QUBO objective (result.fval): 43.937000
QAOA didn't give a clean Hamiltonian cycle; selected patients: [2, 4, 5]
Using brute-force best permutation among selected patients: (1, 2, 3) cost: 36.82
Trip A distance (evaluated): 36.820000, Trip B distance: 31.414000, Total: 68.234000

--- Combination 10/10: Trip A patients = ['R2', 'R3_2', 'IT'] ---
Local problem size n_local=4 nodes -> expected qubits m = n_local*(n_local-1) = 12
QAOA instantiated with args: {'optimizer': <__main__.SciPyOptimizerWrapper object at 0x7f06c9fd56a0>, 'reps': 2}
Running MinimumEigenOptimizer (QAOA) on QUBO (this will call the sampler)...


/usr/local/lib/python3.12/dist-packages/qiskit_optimization/minimum_eigensolvers/qaoa.py:122: UserWarning: Using Sampler V2 (other than StatevectorSampler) without a pass_manager may result in an error. Consider providing a pass_manager for proper circuit transpilation.
  super().__init__(


QAOA solve finished in 1.609 s. Status: OptimizationResultStatus.SUCCESS
Raw QUBO objective (result.fval): 35.965000
Trip A distance (evaluated): 35.965000, Trip B distance: 28.774000, Total: 64.739000
>> New best solution recorded.

FINAL BEST SOLUTION (quantum Trip A):
Total optimal distance (hybrid): 64.739000
Trip A (3 patients): ['R2', 'R3_2', 'IT'] Route: H -> R3_2 -> IT -> R2 Distance: 35.965000
Trip B (2 patients): ['DT', 'GR'] Route: H -> DT -> GR -> H Distance: 28.774000
QAOA sampler: SamplerV2 QAOA time (s): 1.6086161136627197


## Ideal Simulation (more logging)

In [ ]:
# REVISED hybrid QAOA + classical hybrid search with energy logging


!pip install qiskit qiskit-aer qiskit-optimization qiskit_algorithms

import time, itertools, math, sys, traceback
import numpy as np

from qiskit_optimization import QuadraticProgram
from qiskit_optimization.converters import QuadraticProgramToQubo
from qiskit_optimization.algorithms import MinimumEigenOptimizer
from qiskit_optimization.minimum_eigensolvers import QAOA

# Try robust sampler imports
SamplerV2 = None
try:
    from qiskit_aer.primitives import SamplerV2 as _SamplerV2
    SamplerV2 = _SamplerV2
except Exception:
    try:
        from qiskit.primitives import Sampler as _SamplerV2
        SamplerV2 = _SamplerV2
    except Exception:
        SamplerV2 = None

from qiskit_aer import AerSimulator

# optimizer selection (SPSA preferred)
optimizer = None
try:
    from qiskit.algorithms.optimizers import SPSA
    optimizer = SPSA(maxiter=50)
    print("Using SPSA optimizer")
except Exception:
    try:
        from qiskit.algorithms.optimizers import COBYLA
        optimizer = COBYLA(maxiter=200)
        print("Using COBYLA optimizer")
    except Exception:
        from scipy.optimize import minimize as scipy_minimize
        class SciPyOptimizerWrapper:
            def __init__(self, method='COBYLA', options=None):
                self.method = method
                self.options = options or {"maxiter":200}
            def optimize(self, num_vars, objective, initial_point=None):
                def obj_for_scipy(x):
                    val = objective(x)
                    return float(val) if not isinstance(val, (tuple,list)) else float(val[0])
                x0 = initial_point if initial_point is not None else [0.0]*num_vars
                res = scipy_minimize(obj_for_scipy, x0, method=self.method, options=self.options)
                return res.x, float(res.fun), getattr(res, "nfev", None)
            def minimize(self, fun, x0, **kwargs):
                method = kwargs.pop("method", self.method)
                options = kwargs.pop("options", self.options)
                res = scipy_minimize(fun, x0, method=method, options=options, **kwargs)
                return res
        optimizer = SciPyOptimizerWrapper(method='COBYLA', options={"maxiter":200})
        print("Using SciPy-based optimizer wrapper")

# reproducible seeds (helps debugging)
SEED = 1234

# Create sampler / simulator instance robustly
sampler = None
sampler_errors = []
constructors = []
if SamplerV2 is not None:
    # prefer specifying a seeded AerSimulator where possible
    constructors = [
        lambda: SamplerV2(mode=AerSimulator(seed_simulator=SEED, seed_transpiler=SEED)),
        lambda: SamplerV2(backend=AerSimulator(seed_simulator=SEED, seed_transpiler=SEED)),
        lambda: SamplerV2()
    ]
else:
    constructors = [lambda: AerSimulator(seed_simulator=SEED, seed_transpiler=SEED)]

for ctor in constructors:
    try:
        sampler = ctor()
        print("Sampler created with constructor:", ctor)
        break
    except Exception as e:
        sampler_errors.append(e)

if sampler is None:
    print("Sampler constructors tried and failed. Errors:")
    for i, e in enumerate(sampler_errors, 1):
        print(f"[{i}] {type(e).__name__}: {e}")
    raise RuntimeError("Unable to instantiate Sampler/AerSimulator. Ensure qiskit-aer and primitives are installed.")

# ---- Problem data ----
distance_matrix = np.array(
      [[ 0.   ,  8.628, 11.496,  9.445, 10.852,  9.672], # H
       [14.194,  0.   ,  2.361, 10.922,  9.238,  9.43 ], # P1 (DT)
       [17.785,  7.745,  0.   , 11.808, 10.124, 10.478], # P2 (GR)
       [11.864, 19.672, 15.661,  0.   , 11.572, 11.538], # P3 (R2)
       [ 7.343, 12.172, 10.053,  4.071,  0.   ,  5.931], # P4 (R3_2)
       [ 9.269,  9.398, 12.266,  7.318,  8.725,  0.   ]]  # P5 (IT)
)
locations = ['H', 'DT', 'GR', 'R2', 'R3_2', 'IT']
patient_indices = [1,2,3,4,5]
penalty = 1000

# Helper: build TSP QuadraticProgram for a local subproblem (nodes = hospital + chosen patients)
def build_tsp_quadratic_program(dist_local, penalty=penalty):
    n = dist_local.shape[0]
    qp = QuadraticProgram(name="TSP_local")
    for i in range(n):
        for j in range(n):
            if i == j: continue
            qp.binary_var(name=f"x_{i}_{j}")
    linear = {f"x_{i}_{j}": float(dist_local[i,j]) for i in range(n) for j in range(n) if i!=j}
    qp.minimize(linear=linear)
    for i in range(n):
        arrive = {f"x_{j}_{i}": 1 for j in range(n) if j!=i}
        qp.linear_constraint(linear=arrive, sense="==", rhs=1, name=f"arrive_{i}")
        depart = {f"x_{i}_{j}": 1 for j in range(n) if j!=i}
        qp.linear_constraint(linear=depart, sense="==", rhs=1, name=f"depart_{i}")
    qubo = QuadraticProgramToQubo(penalty=penalty).convert(qp)
    return qp, qubo

def reconstruct_tour_from_solution(variables_dict):
    path_dict = {}
    for varname, val in variables_dict.items():
        try:
            if float(val) > 0.5:
                parts = varname.split('_')
                if len(parts) >= 3:
                    si = int(parts[1]); sj = int(parts[2])
                    path_dict[si] = sj
        except Exception:
            continue
    path = [0]
    cur = 0
    while True:
        nxt = path_dict.get(cur, None)
        if nxt is None: break
        if nxt in path:
            break
        path.append(nxt)
        cur = nxt
        if len(path) > 50: break
    if len(path) >= 1 and path[-1] != 0 and len(path) == len(path_dict)+1:
        path.append(0)
    return path

# energy logging structures
loss_history = []
def qaoa_callback(eval_count, parameters, mean, std):
    # This callback matches the qiskit callback signature used in some QAOA wrappers:
    loss_history.append(mean)
    print(f"[QAOA] Iter {eval_count:3d}  Energy (mean) = {mean:.8f}  std = {std if std is not None else 'N/A'}")

# Top-level hybrid search
best = {"distance": float('inf')}
print("Starting hybrid search: Trip A quantum (QAOA) on 3 patients, Trip B classical (2 patients).")

for idx, group_A in enumerate(itertools.combinations(patient_indices, 3), start=1):
    print(f"\n--- Combination {idx}/10: Trip A patients = {[locations[i] for i in group_A]} ---")
    group_B = tuple(p for p in patient_indices if p not in group_A)

    trip_A_nodes = (0,) + group_A
    dist_A_local = distance_matrix[np.ix_(trip_A_nodes, trip_A_nodes)]
    n_local = dist_A_local.shape[0]
    expected_qubits = n_local * (n_local - 1)
    print(f"Local n_local={n_local}, expected binary variables (qubits) = {expected_qubits}")

    qp_local, qubo_local = build_tsp_quadratic_program(dist_A_local, penalty=penalty)

    # instantiate QAOA robustly, include callback and optimizer
    qaoa = None
    qaoa_errors = []
    attempts = [
        {"sampler": sampler, "optimizer": optimizer, "reps": 2, "callback": qaoa_callback},
        {"optimizer": optimizer, "sampler": sampler, "reps": 2, "callback": qaoa_callback},
        {"sampler": sampler, "reps": 2, "optimizer": optimizer, "callback": qaoa_callback},
        {"sampler": sampler, "reps": 2},  # fallback without callback if not supported
    ]
    for kwargs in attempts:
        try:
            qaoa = QAOA(**kwargs)
            print("QAOA instantiated with args:", {k:v for k,v in kwargs.items() if k!='sampler' and k!='callback'})
            break
        except Exception as e:
            qaoa_errors.append((kwargs, e))
    if qaoa is None:
        print("QAOA instantiation failed. Details:")
        for kw, err in qaoa_errors:
            print("Attempt:", {k: (v if k!='sampler' else "<sampler>") for k,v in kw.items()})
            print(" ->", type(err).__name__, err)
        raise RuntimeError("QAOA could not be instantiated in this environment.")

    # Wrap in MinimumEigenOptimizer (some versions accept keyword; handle TypeError)
    try:
        meo = MinimumEigenOptimizer(min_eigen_solver=qaoa)
    except TypeError:
        meo = MinimumEigenOptimizer(qaoa)

    # Run the solver and time it
    print("Running MinimumEigenOptimizer (QAOA) on QUBO...")
    t0 = time.time()
    try:
        result = meo.solve(qubo_local)
    except Exception as e:
        print("Solver failed with exception:")
        traceback.print_exc()
        raise
    t1 = time.time()
    elapsed = t1 - t0
    print(f"QAOA solve finished in {elapsed:.3f} s. result.fval = {getattr(result,'fval', 'N/A')}")

    # Attempt to extract solver diagnostics (robustly)
    print("\n--- QAOA / solver diagnostics ---")
    solver_result = getattr(result, "min_eigen_solver_result", None)
    if solver_result is None:
        # some versions store solver result under 'raw_results' or 'aux_operators_evaluated' - print full object
        print("No min_eigen_solver_result attribute on result. Dumping available attributes:")
        for k in dir(result):
            if not k.startswith('_'):
                try:
                    print(" result.", k, "=", getattr(result, k))
                except Exception:
                    pass
    else:
        # many qiskit minimum_eigensolver results expose eigenvalue, optimal_parameters, aux_operators_evaluated
        try:
            eig = getattr(solver_result, "eigenvalue", None)
            print("Eigenvalue (solver_result.eigenvalue):", eig)
        except Exception:
            pass
        # try common fields
        for field in ("optimal_parameters", "optimal_point", "optimal_parameters_list", "eigenstate", "aux_operators_evaluated"):
            if hasattr(solver_result, field):
                print(f"solver_result.{field}:", getattr(solver_result, field))

    # also print the optimization trace we recorded via callback
    if loss_history:
        print("\nQAOA loss_history (last 10):", loss_history[-10:])
    else:
        print("No loss_history recorded via callback (callback may not have been called).")

    # Reconstruct route from result and evaluate trip A distance
    path_local = reconstruct_tour_from_solution(result.variables_dict)
    if len(path_local) < n_local:
        print("Warning: extracted path is incomplete (may be due to QAOA approximation). Extracted local path:", path_local)

    # compute trip A distance either from path_local or brute-force among selected
    tripA_dist = None
    if len(path_local) == n_local and path_local[0] == 0:
        tour = path_local + ([0] if path_local[-1] != 0 else [])
        tripA_dist = 0.0
        for a,b in zip(tour[:-1], tour[1:]):
            tripA_dist += float(dist_A_local[a,b])
    else:
        # fallback: brute-force best order among selected patients indicated by selected edges
        selected = sorted([j for j in range(1,n_local) if any((f"x_{i}_{j}" in result.variables_dict and result.variables_dict[f"x_{i}_{j}"]>0.5) for i in range(n_local) if i!=j)])
        if not selected:
            raise RuntimeError("QAOA returned no selected patient edges; aborting.")
        best_perm = None; best_perm_cost = math.inf
        for perm in itertools.permutations(selected):
            tour = [0] + list(perm) + [0]
            cost = sum(float(dist_A_local[a,b]) for a,b in zip(tour[:-1], tour[1:]))
            if cost < best_perm_cost:
                best_perm_cost = cost; best_perm = perm
        tripA_dist = best_perm_cost
        path_global = [trip_A_nodes[i] for i in ([0] + list(best_perm) + [0])]
        print("QAOA didn't give a clean Hamiltonian cycle; selected patients:", [trip_A_nodes[i] for i in selected])
        print("Using brute-force best permutation among selected patients:", best_perm, "cost:", best_perm_cost)
    # If earlier we had a full path_local, convert to global
    if len(path_local) == n_local:
        path_global = [trip_A_nodes[i] for i in path_local]
        if path_global[-1] != 0:
            path_global = path_global + [0]

    # --- Trip B classical exact (2 patients) ---
    p1, p2 = group_B
    dist_B_option1 = distance_matrix[0,p1] + distance_matrix[p1,p2] + distance_matrix[p2,0]
    dist_B_option2 = distance_matrix[0,p2] + distance_matrix[p2,p1] + distance_matrix[p1,0]
    if dist_B_option1 <= dist_B_option2:
        tripB_dist = float(dist_B_option1)
        pathB_global = [0, p1, p2, 0]
    else:
        tripB_dist = float(dist_B_option2)
        pathB_global = [0, p2, p1, 0]

    total = float(tripA_dist) + float(tripB_dist)
    print(f"Trip A distance (evaluated): {tripA_dist:.6f}, Trip B distance: {tripB_dist:.6f}, Total: {total:.6f}")

    if total < best["distance"]:
        best.update({
            "distance": total,
            "trip_A_group": [locations[i] for i in group_A],
            "trip_B_group": [locations[i] for i in group_B],
            "trip_A_route": [locations[i] for i in path_global],
            "trip_B_route": [locations[i] for i in pathB_global],
            "trip_A_dist": tripA_dist,
            "trip_B_dist": tripB_dist,
            "qaoa_time": elapsed,
            "qaoa_sampler_constructor": type(sampler).__name__,
            "loss_history_snapshot": loss_history[-20:]
        })
        print(">> New best solution recorded.")

# Final output
print("\n" + "="*50)
print("FINAL BEST SOLUTION (quantum Trip A):")
print("Total optimal distance (hybrid): {:.6f}".format(best["distance"]))
print("Trip A (3 patients):", best["trip_A_group"], "Route:", " -> ".join(best["trip_A_route"]), f"Distance: {best['trip_A_dist']:.6f}")
print("Trip B (2 patients):", best["trip_B_group"], "Route:", " -> ".join(best["trip_B_route"]), f"Distance: {best['trip_B_dist']:.6f}")
print("QAOA sampler:", best.get("qaoa_sampler_constructor"), "QAOA time (s):", best.get("qaoa_time"))
print("="*50)


Using SciPy-based optimizer wrapper
Sampler created with constructor: <function <lambda> at 0x7f069d113060>
Starting hybrid search: Trip A quantum (QAOA) on 3 patients, Trip B classical (2 patients).

--- Combination 1/10: Trip A patients = ['DT', 'GR', 'R2'] ---
Local n_local=4, expected binary variables (qubits) = 12
QAOA instantiated with args: {'optimizer': <__main__.SciPyOptimizerWrapper object at 0x7f06a3453800>, 'reps': 2}
Running MinimumEigenOptimizer (QAOA) on QUBO...
[QAOA] Iter   1  Energy (mean) = 186.68607422+0.00000000j  std = {'shots': 1024, 'circuit_metadata': {}, 'simulator_metadata': {'time_taken_parameter_binding': 0.00013207, 'time_taken_execute': 0.006660458, 'omp_enabled': True, 'max_gpu_memory_mb': 0, 'max_memory_mb': 12977, 'parallel_experiments': 1}}
[QAOA] Iter   2  Energy (mean) = 519.41172266+0.00000000j  std = {'shots': 1024, 'circuit_metadata': {}, 'simulator_metadata': {'time_taken_parameter_binding': 0.000115634, 'time_taken_execute': 0.006613547, 'omp_e

/usr/local/lib/python3.12/dist-packages/qiskit_optimization/minimum_eigensolvers/qaoa.py:122: UserWarning: Using Sampler V2 (other than StatevectorSampler) without a pass_manager may result in an error. Consider providing a pass_manager for proper circuit transpilation.
  super().__init__(


[QAOA] Iter   6  Energy (mean) = 213.57461426+0.00000000j  std = {'shots': 1024, 'circuit_metadata': {}, 'simulator_metadata': {'time_taken_parameter_binding': 0.000115447, 'time_taken_execute': 0.006588177, 'omp_enabled': True, 'max_gpu_memory_mb': 0, 'max_memory_mb': 12977, 'parallel_experiments': 1}}
[QAOA] Iter   7  Energy (mean) = -2890.31883984+0.00000000j  std = {'shots': 1024, 'circuit_metadata': {}, 'simulator_metadata': {'time_taken_parameter_binding': 0.00011479, 'time_taken_execute': 0.006253136, 'omp_enabled': True, 'max_gpu_memory_mb': 0, 'max_memory_mb': 12977, 'parallel_experiments': 1}}
[QAOA] Iter   8  Energy (mean) = 5.55937402+0.00000000j  std = {'shots': 1024, 'circuit_metadata': {}, 'simulator_metadata': {'time_taken_parameter_binding': 0.000116176, 'time_taken_execute': 0.006710766, 'omp_enabled': True, 'max_gpu_memory_mb': 0, 'max_memory_mb': 12977, 'parallel_experiments': 1}}
[QAOA] Iter   9  Energy (mean) = -153.06607617+0.00000000j  std = {'shots': 1024, 'cir

/usr/local/lib/python3.12/dist-packages/qiskit_optimization/minimum_eigensolvers/qaoa.py:122: UserWarning: Using Sampler V2 (other than StatevectorSampler) without a pass_manager may result in an error. Consider providing a pass_manager for proper circuit transpilation.
  super().__init__(


[QAOA] Iter   4  Energy (mean) = -1.94983594+0.00000000j  std = {'shots': 1024, 'circuit_metadata': {}, 'simulator_metadata': {'time_taken_parameter_binding': 0.000166892, 'time_taken_execute': 0.022514355, 'omp_enabled': True, 'max_gpu_memory_mb': 0, 'max_memory_mb': 12977, 'parallel_experiments': 1}}
[QAOA] Iter   5  Energy (mean) = 608.73605664+0.00000000j  std = {'shots': 1024, 'circuit_metadata': {}, 'simulator_metadata': {'time_taken_parameter_binding': 0.000176461, 'time_taken_execute': 0.01235419, 'omp_enabled': True, 'max_gpu_memory_mb': 0, 'max_memory_mb': 12977, 'parallel_experiments': 1}}
[QAOA] Iter   6  Energy (mean) = 805.18444043+0.00000000j  std = {'shots': 1024, 'circuit_metadata': {}, 'simulator_metadata': {'time_taken_parameter_binding': 0.000122354, 'time_taken_execute': 0.006610677, 'omp_enabled': True, 'max_gpu_memory_mb': 0, 'max_memory_mb': 12977, 'parallel_experiments': 1}}
[QAOA] Iter   7  Energy (mean) = 217.63237793+0.00000000j  std = {'shots': 1024, 'circu

...

[QAOA] Iter   3  Energy (mean) = -26.42254492+0.00000000j  std = {'shots': 1024, 'circuit_metadata': {}, 'simulator_metadata': {'time_taken_parameter_binding': 0.000180131, 'time_taken_execute': 0.010528306, 'omp_enabled': True, 'max_gpu_memory_mb': 0, 'max_memory_mb': 12977, 'parallel_experiments': 1}}
[QAOA] Iter   4  Energy (mean) = -253.63210352+0.00000000j  std = {'shots': 1024, 'circuit_metadata': {}, 'simulator_metadata': {'time_taken_parameter_binding': 0.000201775, 'time_taken_execute': 0.009852154, 'omp_enabled': True, 'max_gpu_memory_mb': 0, 'max_memory_mb': 12977, 'parallel_experiments': 1}}
[QAOA] Iter   5  Energy (mean) = -192.34230273+0.00000000j  std = {'shots': 1024, 'circuit_metadata': {}, 'simulator_metadata': {'time_taken_parameter_binding': 0.00011998, 'time_taken_execute': 0.006631393, 'omp_enabled': True, 'max_gpu_memory_mb': 0, 'max_memory_mb': 12977, 'parallel_experiments': 1}}
[QAOA] Iter   6  Energy (mean) = -1182.56334863+0.00000000j  std = {'shots': 1024, '

/usr/local/lib/python3.12/dist-packages/qiskit_optimization/minimum_eigensolvers/qaoa.py:122: UserWarning: Using Sampler V2 (other than StatevectorSampler) without a pass_manager may result in an error. Consider providing a pass_manager for proper circuit transpilation.
  super().__init__(


[QAOA] Iter   6  Energy (mean) = -41.52940820+0.00000000j  std = {'shots': 1024, 'circuit_metadata': {}, 'simulator_metadata': {'time_taken_parameter_binding': 0.000127899, 'time_taken_execute': 0.009107707, 'omp_enabled': True, 'max_gpu_memory_mb': 0, 'max_memory_mb': 12977, 'parallel_experiments': 1}}
[QAOA] Iter   7  Energy (mean) = -148.29766309+0.00000000j  std = {'shots': 1024, 'circuit_metadata': {}, 'simulator_metadata': {'time_taken_parameter_binding': 0.000117977, 'time_taken_execute': 0.006811381, 'omp_enabled': True, 'max_gpu_memory_mb': 0, 'max_memory_mb': 12977, 'parallel_experiments': 1}}
[QAOA] Iter   8  Energy (mean) = -1139.15177344+0.00000000j  std = {'shots': 1024, 'circuit_metadata': {}, 'simulator_metadata': {'time_taken_parameter_binding': 0.000115601, 'time_taken_execute': 0.00658486, 'omp_enabled': True, 'max_gpu_memory_mb': 0, 'max_memory_mb': 12977, 'parallel_experiments': 1}}
[QAOA] Iter   9  Energy (mean) = -101.75749609+0.00000000j  std = {'shots': 1024, '

/usr/local/lib/python3.12/dist-packages/qiskit_optimization/minimum_eigensolvers/qaoa.py:122: UserWarning: Using Sampler V2 (other than StatevectorSampler) without a pass_manager may result in an error. Consider providing a pass_manager for proper circuit transpilation.
  super().__init__(


[QAOA] Iter   6  Energy (mean) = 2987.80144141+0.00000000j  std = {'shots': 1024, 'circuit_metadata': {}, 'simulator_metadata': {'time_taken_parameter_binding': 0.000114586, 'time_taken_execute': 0.007015623, 'omp_enabled': True, 'max_gpu_memory_mb': 0, 'max_memory_mb': 12977, 'parallel_experiments': 1}}
[QAOA] Iter   7  Energy (mean) = -697.91557129+0.00000000j  std = {'shots': 1024, 'circuit_metadata': {}, 'simulator_metadata': {'time_taken_parameter_binding': 0.000125099, 'time_taken_execute': 0.00642071, 'omp_enabled': True, 'max_gpu_memory_mb': 0, 'max_memory_mb': 12977, 'parallel_experiments': 1}}
[QAOA] Iter   8  Energy (mean) = -12.08595410+0.00000000j  std = {'shots': 1024, 'circuit_metadata': {}, 'simulator_metadata': {'time_taken_parameter_binding': 0.000121177, 'time_taken_execute': 0.006646245, 'omp_enabled': True, 'max_gpu_memory_mb': 0, 'max_memory_mb': 12977, 'parallel_experiments': 1}}
[QAOA] Iter   9  Energy (mean) = 227.65085059+0.00000000j  std = {'shots': 1024, 'ci

## Noisy Simulation

In [ ]:
# Robust cell: FakeProvider noise -> AerSimulator -> primitives Sampler -> QAOA + MinimumEigenOptimizer


import time, itertools, math, traceback
import numpy as np

# Qiskit optimization pieces
from qiskit_optimization import QuadraticProgram
from qiskit_optimization.converters import QuadraticProgramToQubo
from qiskit_optimization.algorithms import MinimumEigenOptimizer
from qiskit_optimization.minimum_eigensolvers import QAOA

# Fake provider + Aer noise
from qiskit.providers.fake_provider import GenericBackendV2
from qiskit_aer import AerSimulator
from qiskit_aer.noise import NoiseModel

# Try to find a primitives-style Sampler implementation (many Qiskit versions expose one)
PrimitiveSamplerClass = None
sampler_creation_exceptions = []

# Try qiskit.primitives.Sampler first (common)
try:
    from qiskit.primitives import Sampler as _SamplerPrimitive
    PrimitiveSamplerClass = _SamplerPrimitive
    print("Found qiskit.primitives.Sampler")
except Exception as e:
    sampler_creation_exceptions.append(("qiskit.primitives.Sampler import", e))

# Try qiskit_aer.primitives.SamplerV2 next
if PrimitiveSamplerClass is None:
    try:
        from qiskit_aer.primitives import SamplerV2 as _SamplerPrimitiveV2
        PrimitiveSamplerClass = _SamplerPrimitiveV2
        print("Found qiskit_aer.primitives.SamplerV2")
    except Exception as e:
        sampler_creation_exceptions.append(("qiskit_aer.primitives.SamplerV2 import", e))

# As a last attempt, try qiskit_aer.primitives.Sampler (older naming)
if PrimitiveSamplerClass is None:
    try:
        from qiskit_aer.primitives import Sampler as _SamplerPrimitiveAlt
        PrimitiveSamplerClass = _SamplerPrimitiveAlt
        print("Found qiskit_aer.primitives.Sampler (alt)")
    except Exception as e:
        sampler_creation_exceptions.append(("qiskit_aer.primitives.Sampler import", e))



# Optimizer selection / SciPy fallback (keeps minimize() signature used by Qiskit)
optimizer = None
try:
    from qiskit.algorithms.optimizers import SPSA
    optimizer = SPSA(maxiter=50)
    print("Using SPSA optimizer.")
except Exception:
    try:
        from qiskit.algorithms.optimizers import COBYLA
        optimizer = COBYLA(maxiter=200)
        print("Using COBYLA optimizer.")
    except Exception:
        # SciPy-based wrapper (implements minimize compatible with Qiskit)
        from scipy.optimize import minimize as scipy_minimize
        class SciPyOptimizerWrapper:
            def __init__(self, method='COBYLA', options=None):
                self.method = method
                self.options = options or {"maxiter":200}
            def optimize(self, num_vars, objective, initial_point=None):
                def obj_for_scipy(x):
                    val = objective(x)
                    return float(val) if not isinstance(val, (tuple,list)) else float(val[0])
                x0 = initial_point if initial_point is not None else [0.0]*num_vars
                res = scipy_minimize(obj_for_scipy, x0, method=self.method, options=self.options)
                return res.x, float(res.fun), getattr(res, "nfev", None)
            def minimize(self, fun, x0, **kwargs):
                method = kwargs.pop("method", self.method)
                options = kwargs.pop("options", self.options)
                res = scipy_minimize(fun, x0, method=method, options=options, **kwargs)
                return res
        optimizer = SciPyOptimizerWrapper(method='COBYLA', options={"maxiter":100})
        print("Using SciPy-based optimizer wrapper.")

# ------------------------------
# Problem data 
# ------------------------------
distance_matrix = np.array(
      [[ 0.   ,  8.628, 11.496,  9.445, 10.852,  9.672],
       [14.194,  0.   ,  2.361, 10.922,  9.238,  9.43 ],
       [17.785,  7.745,  0.   , 11.808, 10.124, 10.478],
       [11.864, 19.672, 15.661,  0.   , 11.572, 11.538],
       [ 7.343, 12.172, 10.053,  4.071,  0.   ,  5.931],
       [ 9.269,  9.398, 12.266,  7.318,  8.725,  0.   ]]
)
locations = ['H', 'DT', 'GR', 'R2', 'R3_2', 'IT']
patient_indices = [1,2,3,4,5]
penalty = 1000

# Build fake backend & noise model
max_n_local = 4
expected_qubits = max_n_local * (max_n_local - 1)
print("Creating GenericBackendV2 with", expected_qubits, "qubits (for noise model).")
fake_backend = GenericBackendV2(num_qubits=expected_qubits)
try:
    # try using QasmSimulator.from_backend if available 
    from qiskit_aer import QasmSimulator
    try:
        device_for_noise = QasmSimulator.from_backend(fake_backend)
        noise_model = NoiseModel.from_backend(device_for_noise)
    except Exception:
        noise_model = NoiseModel.from_backend(fake_backend)
except Exception:
    noise_model = NoiseModel.from_backend(fake_backend)

print("Noise model created from fake backend.")

# Build noisy AerSimulator defensively (avoid passing unsupported kwargs)
sim_seed = 1234
simulator = None
try:
    simulator = AerSimulator(noise_model=noise_model, method='density_matrix', seed_simulator=sim_seed)
    print("AerSimulator created with method='density_matrix' and seed_simulator.")
except Exception as e1:
    print("Preferred AerSimulator constructor failed:", type(e1).__name__, e1)
    try:
        simulator = AerSimulator(noise_model=noise_model, method='density_matrix')
        print("AerSimulator created with method='density_matrix' (no seed).")
    except Exception:
        try:
            simulator = AerSimulator(noise_model=noise_model)
            print("AerSimulator created with noise_model (default method).")
        except Exception as e_last:
            print("Failed to create AerSimulator with noise_model. Exception:")
            traceback.print_exc()
            raise RuntimeError("Cannot create AerSimulator in this environment.") from e_last

# Now create a primitives-style sampler backed by the simulator
sampler = None
sampler_errors = []

if PrimitiveSamplerClass is not None:
    # try several common constructor signatures
    constructors = [
        lambda cls=PrimitiveSamplerClass: cls(backend=simulator),
        lambda cls=PrimitiveSamplerClass: cls(backend=simulator, shots=1024),
        lambda cls=PrimitiveSamplerClass: cls(backend=simulator, mode=simulator),
        lambda cls=PrimitiveSamplerClass: cls(mode=simulator),
        lambda cls=PrimitiveSamplerClass: cls(),
    ]
    for ctor in constructors:
        try:
            sampler = ctor()
            print("Primitive sampler created with constructor:", ctor)
            break
        except Exception as e:
            sampler_errors.append((ctor, e))
else:
    sampler_errors.append(("PrimitiveSamplerClass not found", sampler_creation_exceptions))

# If still failed -> helpful error message
if sampler is None:
    print("Failed to create a primitives-style Sampler. Attempts and exceptions:")
    for c, e in sampler_errors:
        print("-", c, "raised:", repr(e))
    raise RuntimeError(
        "Unable to instantiate a primitives Sampler compatible with this Qiskit version. "
        "Please ensure you have a primitives Sampler implementation available (e.g. install qiskit-aer and try importing qiskit.primitives.Sampler or qiskit_aer.primitives.SamplerV2)."
    )

print("Using sampler object of type:", type(sampler).__name__)

# ---- QuadraticProgram helper and reconstruction ----
def build_tsp_quadratic_program(dist_local, penalty=penalty):
    n = dist_local.shape[0]
    qp = QuadraticProgram(name="TSP_local")
    for i in range(n):
        for j in range(n):
            if i == j: continue
            qp.binary_var(name=f"x_{i}_{j}")
    linear = {f"x_{i}_{j}": float(dist_local[i,j]) for i in range(n) for j in range(n) if i!=j}
    qp.minimize(linear=linear)
    for i in range(n):
        arrive = {f"x_{j}_{i}": 1 for j in range(n) if j!=i}
        qp.linear_constraint(linear=arrive, sense="==", rhs=1, name=f"arrive_{i}")
        depart = {f"x_{i}_{j}": 1 for j in range(n) if j!=i}
        qp.linear_constraint(linear=depart, sense="==", rhs=1, name=f"depart_{i}")
    qubo = QuadraticProgramToQubo(penalty=penalty).convert(qp)
    return qp, qubo

def reconstruct_tour_from_solution(variables_dict):
    path_dict = {}
    for varname, val in variables_dict.items():
        try:
            if float(val) > 0.5:
                parts = varname.split('_')
                if len(parts) >= 3:
                    si = int(parts[1]); sj = int(parts[2])
                    path_dict[si] = sj
        except Exception:
            continue
    path = [0]
    cur = 0
    while True:
        nxt = path_dict.get(cur, None)
        if nxt is None: break
        if nxt in path: break
        path.append(nxt); cur = nxt
        if len(path) > 50: break
    if len(path) >= 1 and path[-1] != 0 and len(path) == len(path_dict)+1:
        path.append(0)
    return path

# QAOA callback & loss history
loss_history = []
def qaoa_callback(eval_count, parameters, mean, std):
    loss_history.append(mean)
    print(f"[QAOA] Iter {eval_count:3d}  Energy = {mean:.8f}  std = {std if std is not None else 'N/A'}")

# Top-level hybrid search (Trip A quantum on 3 patients; Trip B classical on remaining 2)
best = {"distance": float('inf')}

print("Starting hybrid search using noisy sampler (FakeProvider -> AerSimulator -> primitives Sampler).")

for idx, group_A in enumerate(itertools.combinations(patient_indices, 3), start=1):
    print(f"\n--- Combination {idx}/10: Trip A patients = {[locations[i] for i in group_A]} ---")
    group_B = tuple(p for p in patient_indices if p not in group_A)

    trip_A_nodes = (0,) + group_A
    dist_A_local = distance_matrix[np.ix_(trip_A_nodes, trip_A_nodes)]
    n_local = dist_A_local.shape[0]
    print(f"Local problem size n_local={n_local} nodes -> expected binary vars = {n_local*(n_local-1)}")

    qp_local, qubo_local = build_tsp_quadratic_program(dist_A_local, penalty=penalty)

    # Instantiate QAOA robustly (use sampler + optimizer)
    qaoa = None
    qaoa_errors = []
    attempts = [
        {"sampler": sampler, "optimizer": optimizer, "reps": 2, "callback": qaoa_callback},
        {"optimizer": optimizer, "sampler": sampler, "reps": 2, "callback": qaoa_callback},
        {"sampler": sampler, "reps": 2, "optimizer": optimizer, "callback": qaoa_callback},
        {"sampler": sampler, "reps": 2, "optimizer": optimizer},
    ]
    for kwargs in attempts:
        try:
            qaoa = QAOA(**kwargs)
            print("QAOA instantiated with args:", {k:v for k,v in kwargs.items() if k not in ('sampler','callback')})
            break
        except Exception as e:
            qaoa_errors.append((kwargs, e))

    if qaoa is None:
        print("QAOA instantiation failed. Details:")
        for kw, err in qaoa_errors:
            print("Attempt:", {k: (v if k!='sampler' else "<sampler>") for k,v in kw.items()})
            print(" ->", type(err).__name__, err)
        raise RuntimeError("QAOA could not be instantiated.")

    # Wrap in MinimumEigenOptimizer (handle older/newer constructors)
    try:
        meo = MinimumEigenOptimizer(min_eigen_solver=qaoa)
    except TypeError:
        meo = MinimumEigenOptimizer(qaoa)

    print("Running MinimumEigenOptimizer (QAOA) on noisy QUBO ...")
    t0 = time.time()
    try:
        result = meo.solve(qubo_local)
    except Exception:
        traceback.print_exc()
        raise
    t1 = time.time()
    print(f"QAOA solve finished in {t1-t0:.3f} s. result.fval = {getattr(result, 'fval', 'N/A')}")

    # Diagnostics
    solver_result = getattr(result, "min_eigen_solver_result", None)
    if solver_result is None:
        print("No min_eigen_solver_result found on result; printing basic attributes:")
        for k in ['fval','status','variables_dict']:
            if hasattr(result, k):
                print(" result.", k, "=", getattr(result, k))
    else:
        for field in ("eigenvalue", "optimal_parameters", "optimal_point", "eigenstate", "aux_operators_evaluated"):
            if hasattr(solver_result, field):
                print(f"solver_result.{field}:", getattr(solver_result, field))

    if loss_history:
        print("Recent loss_history (last 10):", loss_history[-10:])
    else:
        print("No loss_history recorded (callback may not have been called).")

    # Reconstruct route and evaluate trip A distance
    path_local = reconstruct_tour_from_solution(result.variables_dict)
    if len(path_local) < n_local:
        print("Warning: extracted path incomplete:", path_local)

    if len(path_local) == n_local and path_local[0] == 0:
        tour = path_local + ([0] if path_local[-1] != 0 else [])
        tripA_dist = sum(float(dist_A_local[a,b]) for a,b in zip(tour[:-1], tour[1:]))
        path_global = [trip_A_nodes[i] for i in path_local]
        if path_global[-1] != 0:
            path_global = path_global + [0]
    else:
        selected = sorted([j for j in range(1,n_local) if any((f"x_{i}_{j}" in result.variables_dict and result.variables_dict[f"x_{i}_{j}"]>0.5) for i in range(n_local) if i!=j)])
        if not selected:
            raise RuntimeError("QAOA returned no selected patient edges; aborting.")
        best_perm = None; best_perm_cost = math.inf
        for perm in itertools.permutations(selected):
            tour = [0] + list(perm) + [0]
            cost = sum(float(dist_A_local[a,b]) for a,b in zip(tour[:-1], tour[1:]))
            if cost < best_perm_cost:
                best_perm_cost = cost; best_perm = perm
        tripA_dist = best_perm_cost
        path_global = [trip_A_nodes[i] for i in ([0] + list(best_perm) + [0])]
        print("Fallback brute-force chosen subset:", [trip_A_nodes[i] for i in selected], "best perm:", best_perm)

    # Trip B classical exact (2 patients)
    p1, p2 = group_B
    dist_B_option1 = distance_matrix[0,p1] + distance_matrix[p1,p2] + distance_matrix[p2,0]
    dist_B_option2 = distance_matrix[0,p2] + distance_matrix[p2,p1] + distance_matrix[p1,0]
    if dist_B_option1 <= dist_B_option2:
        tripB_dist = float(dist_B_option1); pathB_global = [0, p1, p2, 0]
    else:
        tripB_dist = float(dist_B_option2); pathB_global = [0, p2, p1, 0]

    total = float(tripA_dist) + float(tripB_dist)
    print(f"Trip A distance: {tripA_dist:.6f}, Trip B distance: {tripB_dist:.6f}, Total: {total:.6f}")

    if total < best["distance"]:
        best.update({
            "distance": total,
            "trip_A_group": [locations[i] for i in group_A],
            "trip_B_group": [locations[i] for i in group_B],
            "trip_A_route": [locations[i] for i in path_global],
            "trip_B_route": [locations[i] for i in pathB_global],
            "trip_A_dist": tripA_dist,
            "trip_B_dist": tripB_dist,
            "qaoa_time": t1-t0,
            "qaoa_sampler_constructor": type(sampler).__name__
        })
        print(">> New best solution recorded.")

# Final results
print("\n" + "="*50)
print("FINAL BEST SOLUTION (quantum Trip A, noisy simulator):")
print("Total optimal distance (hybrid): {:.6f}".format(best["distance"]))
print("Trip A (3 patients):", best["trip_A_group"], "Route:", " -> ".join(best["trip_A_route"]), f"Distance: {best['trip_A_dist']:.6f}")
print("Trip B (2 patients):", best["trip_B_group"], "Route:", " -> ".join(best["trip_B_route"]), f"Distance: {best['trip_B_dist']:.6f}")
print("QAOA sampler:", best.get("qaoa_sampler_constructor"), "QAOA time (s):", best.get("qaoa_time"))
print("="*50)


Found qiskit_aer.primitives.SamplerV2
Using SciPy-based optimizer wrapper.
Creating GenericBackendV2 with 12 qubits (for noise model).
Noise model created from fake backend.
AerSimulator created with method='density_matrix' and seed_simulator.
Primitive sampler created with constructor: <function <lambda> at 0x7f068ee43d80>
Using sampler object of type: SamplerV2
Starting hybrid search using noisy sampler (FakeProvider -> AerSimulator -> primitives Sampler).

--- Combination 1/10: Trip A patients = ['DT', 'GR', 'R2'] ---
Local problem size n_local=4 nodes -> expected binary vars = 12
QAOA instantiated with args: {'optimizer': <__main__.SciPyOptimizerWrapper object at 0x7f069c4c24e0>, 'reps': 2}
Running MinimumEigenOptimizer (QAOA) on noisy QUBO ...
[QAOA] Iter   1  Energy = 373.38925098+0.00000000j  std = {'shots': 1024, 'circuit_metadata': {}, 'simulator_metadata': {'time_taken_parameter_binding': 0.000118242, 'time_taken_execute': 0.006716816, 'omp_enabled': True, 'max_gpu_memory_mb'

/usr/local/lib/python3.12/dist-packages/qiskit_optimization/minimum_eigensolvers/qaoa.py:122: UserWarning: Using Sampler V2 (other than StatevectorSampler) without a pass_manager may result in an error. Consider providing a pass_manager for proper circuit transpilation.
  super().__init__(


[QAOA] Iter   6  Energy = -440.25929688+0.00000000j  std = {'shots': 1024, 'circuit_metadata': {}, 'simulator_metadata': {'time_taken_parameter_binding': 0.000117472, 'time_taken_execute': 0.006641965, 'omp_enabled': True, 'max_gpu_memory_mb': 0, 'max_memory_mb': 12977, 'parallel_experiments': 1}}
[QAOA] Iter   7  Energy = -327.91006543+0.00000000j  std = {'shots': 1024, 'circuit_metadata': {}, 'simulator_metadata': {'time_taken_parameter_binding': 0.000138742, 'time_taken_execute': 0.006737952, 'omp_enabled': True, 'max_gpu_memory_mb': 0, 'max_memory_mb': 12977, 'parallel_experiments': 1}}
[QAOA] Iter   8  Energy = -388.76979297+0.00000000j  std = {'shots': 1024, 'circuit_metadata': {}, 'simulator_metadata': {'time_taken_parameter_binding': 0.000131477, 'time_taken_execute': 0.006669629, 'omp_enabled': True, 'max_gpu_memory_mb': 0, 'max_memory_mb': 12977, 'parallel_experiments': 1}}
[QAOA] Iter   9  Energy = 159.67464746+0.00000000j  std = {'shots': 1024, 'circuit_metadata': {}, 'simu

/usr/local/lib/python3.12/dist-packages/qiskit_optimization/minimum_eigensolvers/qaoa.py:122: UserWarning: Using Sampler V2 (other than StatevectorSampler) without a pass_manager may result in an error. Consider providing a pass_manager for proper circuit transpilation.
  super().__init__(


[QAOA] Iter   6  Energy = -299.46635254+0.00000000j  std = {'shots': 1024, 'circuit_metadata': {}, 'simulator_metadata': {'time_taken_parameter_binding': 0.000114359, 'time_taken_execute': 0.00659552, 'omp_enabled': True, 'max_gpu_memory_mb': 0, 'max_memory_mb': 12977, 'parallel_experiments': 1}}
[QAOA] Iter   7  Energy = -1145.16175879+0.00000000j  std = {'shots': 1024, 'circuit_metadata': {}, 'simulator_metadata': {'time_taken_parameter_binding': 0.00011404, 'time_taken_execute': 0.006738057, 'omp_enabled': True, 'max_gpu_memory_mb': 0, 'max_memory_mb': 12977, 'parallel_experiments': 1}}
[QAOA] Iter   8  Energy = -15.70459082+0.00000000j  std = {'shots': 1024, 'circuit_metadata': {}, 'simulator_metadata': {'time_taken_parameter_binding': 0.000128009, 'time_taken_execute': 0.006965246, 'omp_enabled': True, 'max_gpu_memory_mb': 0, 'max_memory_mb': 12977, 'parallel_experiments': 1}}
[QAOA] Iter   9  Energy = 2501.57650781+0.00000000j  std = {'shots': 1024, 'circuit_metadata': {}, 'simul

...

[QAOA] Iter   6  Energy = 2318.80665039+0.00000000j  std = {'shots': 1024, 'circuit_metadata': {}, 'simulator_metadata': {'time_taken_parameter_binding': 0.000111503, 'time_taken_execute': 0.006673053, 'omp_enabled': True, 'max_gpu_memory_mb': 0, 'max_memory_mb': 12977, 'parallel_experiments': 1}}
[QAOA] Iter   7  Energy = 242.16813574+0.00000000j  std = {'shots': 1024, 'circuit_metadata': {}, 'simulator_metadata': {'time_taken_parameter_binding': 0.000121292, 'time_taken_execute': 0.006746373, 'omp_enabled': True, 'max_gpu_memory_mb': 0, 'max_memory_mb': 12977, 'parallel_experiments': 1}}
[QAOA] Iter   8  Energy = -669.26049707+0.00000000j  std = {'shots': 1024, 'circuit_metadata': {}, 'simulator_metadata': {'time_taken_parameter_binding': 0.000121457, 'time_taken_execute': 0.00668462, 'omp_enabled': True, 'max_gpu_memory_mb': 0, 'max_memory_mb': 12977, 'parallel_experiments': 1}}
[QAOA] Iter   9  Energy = -1216.65647754+0.00000000j  std = {'shots': 1024, 'circuit_metadata': {}, 'simu

/usr/local/lib/python3.12/dist-packages/qiskit_optimization/minimum_eigensolvers/qaoa.py:122: UserWarning: Using Sampler V2 (other than StatevectorSampler) without a pass_manager may result in an error. Consider providing a pass_manager for proper circuit transpilation.
  super().__init__(


[QAOA] Iter   5  Energy = 261.37027441+0.00000000j  std = {'shots': 1024, 'circuit_metadata': {}, 'simulator_metadata': {'time_taken_parameter_binding': 0.000115014, 'time_taken_execute': 0.008680115, 'omp_enabled': True, 'max_gpu_memory_mb': 0, 'max_memory_mb': 12977, 'parallel_experiments': 1}}
[QAOA] Iter   6  Energy = 47.40462988+0.00000000j  std = {'shots': 1024, 'circuit_metadata': {}, 'simulator_metadata': {'time_taken_parameter_binding': 0.000202497, 'time_taken_execute': 0.009220502, 'omp_enabled': True, 'max_gpu_memory_mb': 0, 'max_memory_mb': 12977, 'parallel_experiments': 1}}
[QAOA] Iter   7  Energy = 1144.28460449+0.00000000j  std = {'shots': 1024, 'circuit_metadata': {}, 'simulator_metadata': {'time_taken_parameter_binding': 0.00011256, 'time_taken_execute': 0.006507307, 'omp_enabled': True, 'max_gpu_memory_mb': 0, 'max_memory_mb': 12977, 'parallel_experiments': 1}}
[QAOA] Iter   8  Energy = -788.51059082+0.00000000j  std = {'shots': 1024, 'circuit_metadata': {}, 'simulat

/usr/local/lib/python3.12/dist-packages/qiskit_optimization/minimum_eigensolvers/qaoa.py:122: UserWarning: Using Sampler V2 (other than StatevectorSampler) without a pass_manager may result in an error. Consider providing a pass_manager for proper circuit transpilation.
  super().__init__(


[QAOA] Iter   6  Energy = 3.35500293+0.00000000j  std = {'shots': 1024, 'circuit_metadata': {}, 'simulator_metadata': {'time_taken_parameter_binding': 0.000119312, 'time_taken_execute': 0.006837155, 'omp_enabled': True, 'max_gpu_memory_mb': 0, 'max_memory_mb': 12977, 'parallel_experiments': 1}}
[QAOA] Iter   7  Energy = -7.87304395+0.00000000j  std = {'shots': 1024, 'circuit_metadata': {}, 'simulator_metadata': {'time_taken_parameter_binding': 0.000201679, 'time_taken_execute': 0.00774369, 'omp_enabled': True, 'max_gpu_memory_mb': 0, 'max_memory_mb': 12977, 'parallel_experiments': 1}}
[QAOA] Iter   8  Energy = -358.36241211+0.00000000j  std = {'shots': 1024, 'circuit_metadata': {}, 'simulator_metadata': {'time_taken_parameter_binding': 0.000112901, 'time_taken_execute': 0.006634816, 'omp_enabled': True, 'max_gpu_memory_mb': 0, 'max_memory_mb': 12977, 'parallel_experiments': 1}}
[QAOA] Iter   9  Energy = -37.37767187+0.00000000j  std = {'shots': 1024, 'circuit_metadata': {}, 'simulator_

## Noisy simulation but with a clearer logging

In [ ]:
# Clean prints version: FakeProvider noise -> AerSimulator -> primitives Sampler -> QAOA + MinimumEigenOptimizer
# Toggle VERBOSE for more internal debug prints (constructor errors, solver_result fields, etc.)

import time, itertools, math, traceback
import numpy as np

VERBOSE = False  # set True to see more debug output

# small helper for controlled printing
def info(*args, **kwargs):
    print(*args, **kwargs)

def dbg(*args, **kwargs):
    if VERBOSE:
        print(*args, **kwargs)

# Qiskit optimization pieces
from qiskit_optimization import QuadraticProgram
from qiskit_optimization.converters import QuadraticProgramToQubo
from qiskit_optimization.algorithms import MinimumEigenOptimizer
from qiskit_optimization.minimum_eigensolvers import QAOA

# Fake provider + Aer noise
from qiskit.providers.fake_provider import GenericBackendV2
from qiskit_aer import AerSimulator
from qiskit_aer.noise import NoiseModel

# Try to find a primitives-style Sampler implementation
PrimitiveSamplerClass = None
sampler_creation_exceptions = []

for attempt_name, attempt_import in [
    ("qiskit.primitives.Sampler", lambda: __import__("qiskit.primitives", fromlist=["Sampler"]).Sampler),
    ("qiskit_aer.primitives.SamplerV2", lambda: __import__("qiskit_aer.primitives", fromlist=["SamplerV2"]).SamplerV2),
    ("qiskit_aer.primitives.Sampler", lambda: __import__("qiskit_aer.primitives", fromlist=["Sampler"]).Sampler),
]:
    try:
        cls = attempt_import()
        PrimitiveSamplerClass = cls
        dbg(f"Primitive sampler available via: {attempt_name}")
        break
    except Exception as e:
        sampler_creation_exceptions.append((attempt_name, e))
        dbg(f"Sampler import {attempt_name} failed: {type(e).__name__}: {e}")

# Optimizer selection / SciPy fallback
optimizer = None
try:
    from qiskit.algorithms.optimizers import SPSA
    optimizer = SPSA(maxiter=50)
    dbg("Using SPSA optimizer.")
except Exception:
    try:
        from qiskit.algorithms.optimizers import COBYLA
        optimizer = COBYLA(maxiter=200)
        dbg("Using COBYLA optimizer.")
    except Exception:
        # SciPy wrapper compatible with QAOA expectations
        from scipy.optimize import minimize as scipy_minimize
        class SciPyOptimizerWrapper:
            def __init__(self, method='COBYLA', options=None):
                self.method = method
                self.options = options or {"maxiter":200}
            def optimize(self, num_vars, objective, initial_point=None):
                def obj_for_scipy(x):
                    val = objective(x)
                    return float(val) if not isinstance(val, (tuple,list)) else float(val[0])
                x0 = initial_point if initial_point is not None else [0.0]*num_vars
                res = scipy_minimize(obj_for_scipy, x0, method=self.method, options=self.options)
                return res.x, float(res.fun), getattr(res, "nfev", None)
            def minimize(self, fun, x0, **kwargs):
                method = kwargs.pop("method", self.method)
                options = kwargs.pop("options", self.options)
                res = scipy_minimize(fun, x0, method=method, options=options, **kwargs)
                return res
        optimizer = SciPyOptimizerWrapper(method='COBYLA', options={"maxiter":100})
        dbg("Using SciPy-based optimizer wrapper.")

# ------------------------------
# Problem data (same as before)
# ------------------------------
distance_matrix = np.array(
      [[ 0.   ,  8.628, 11.496,  9.445, 10.852,  9.672],
       [14.194,  0.   ,  2.361, 10.922,  9.238,  9.43 ],
       [17.785,  7.745,  0.   , 11.808, 10.124, 10.478],
       [11.864, 19.672, 15.661,  0.   , 11.572, 11.538],
       [ 7.343, 12.172, 10.053,  4.071,  0.   ,  5.931],
       [ 9.269,  9.398, 12.266,  7.318,  8.725,  0.   ]]
)
locations = ['H', 'DT', 'GR', 'R2', 'R3_2', 'IT']
patient_indices = [1,2,3,4,5]
penalty = 1000

# Build fake backend & noise model
max_n_local = 4
expected_qubits = max_n_local * (max_n_local - 1)
info(f"Setup: FakeProvider GenericBackendV2 -> noise model with {expected_qubits} qubits (for Trip A up to 3 patients).")
fake_backend = GenericBackendV2(num_qubits=expected_qubits)

try:
    from qiskit_aer import QasmSimulator
    try:
        device_for_noise = QasmSimulator.from_backend(fake_backend)
        noise_model = NoiseModel.from_backend(device_for_noise)
    except Exception:
        noise_model = NoiseModel.from_backend(fake_backend)
except Exception:
    noise_model = NoiseModel.from_backend(fake_backend)
dbg("Noise model created from fake backend.")

# Create AerSimulator defensively (avoid unsupported kwargs)
sim_seed = 1234
simulator = None
try:
    simulator = AerSimulator(noise_model=noise_model, method='density_matrix', seed_simulator=sim_seed)
    dbg("AerSimulator(method='density_matrix', seed_simulator) created.")
except Exception as e1:
    dbg("Preferred AerSimulator constructor failed:", type(e1).__name__, e1)
    try:
        simulator = AerSimulator(noise_model=noise_model, method='density_matrix')
        dbg("AerSimulator(method='density_matrix') created.")
    except Exception:
        simulator = AerSimulator(noise_model=noise_model)
        dbg("AerSimulator(noise_model) created.")

# Build primitives sampler (try a few constructors) or raise helpful error
sampler = None
sampler_errors = []

if PrimitiveSamplerClass is not None:
    constructors = [
        lambda cls=PrimitiveSamplerClass: cls(backend=simulator),
        lambda cls=PrimitiveSamplerClass: cls(backend=simulator, shots=1024),
        lambda cls=PrimitiveSamplerClass: cls(backend=simulator, mode=simulator),
        lambda cls=PrimitiveSamplerClass: cls(mode=simulator),
        lambda cls=PrimitiveSamplerClass: cls(),
    ]
    for ctor in constructors:
        try:
            sampler = ctor()
            dbg("Primitive sampler created with constructor:", ctor)
            break
        except Exception as e:
            sampler_errors.append((ctor, e))
else:
    sampler_errors.append(("PrimitiveSamplerClass not found", sampler_creation_exceptions))

if sampler is None:
    # give a short, actionable error
    info("ERROR: Could not instantiate a primitives-style Sampler compatible with this Qiskit.")
    if VERBOSE:
        for label, ex in sampler_errors:
            dbg("Attempt:", label, "->", repr(ex))
    raise RuntimeError(
        "Install or enable a primitives Sampler (qiskit.primitives.Sampler or qiskit_aer.primitives.SamplerV2)."
    )

info(f"Using sampler: {type(sampler).__name__}")

# ---- QuadraticProgram helper and reconstruction ----
def build_tsp_quadratic_program(dist_local, penalty=penalty):
    n = dist_local.shape[0]
    qp = QuadraticProgram(name="TSP_local")
    for i in range(n):
        for j in range(n):
            if i == j: continue
            qp.binary_var(name=f"x_{i}_{j}")
    linear = {f"x_{i}_{j}": float(dist_local[i,j]) for i in range(n) for j in range(n) if i!=j}
    qp.minimize(linear=linear)
    for i in range(n):
        arrive = {f"x_{j}_{i}": 1 for j in range(n) if j!=i}
        qp.linear_constraint(linear=arrive, sense="==", rhs=1, name=f"arrive_{i}")
        depart = {f"x_{i}_{j}": 1 for j in range(n) if j!=i}
        qp.linear_constraint(linear=depart, sense="==", rhs=1, name=f"depart_{i}")
    qubo = QuadraticProgramToQubo(penalty=penalty).convert(qp)
    return qp, qubo

def reconstruct_tour_from_solution(variables_dict):
    path_dict = {}
    for varname, val in variables_dict.items():
        try:
            if float(val) > 0.5:
                parts = varname.split('_')
                if len(parts) >= 3:
                    si = int(parts[1]); sj = int(parts[2])
                    path_dict[si] = sj
        except Exception:
            continue
    path = [0]; cur = 0
    while True:
        nxt = path_dict.get(cur, None)
        if nxt is None: break
        if nxt in path: break
        path.append(nxt); cur = nxt
        if len(path) > 50: break
    if len(path) >= 1 and path[-1] != 0 and len(path) == len(path_dict)+1:
        path.append(0)
    return path

# QAOA callback & loss history (quiet by default)
loss_history = []
def qaoa_callback(eval_count, parameters, mean, std):
    loss_history.append(mean)
    dbg(f"[QAOA cb] Iter {eval_count}: Energy={mean:.6f} std={std}")

# Top-level hybrid search (Trip A quantum on 3 patients; Trip B classical on remaining 2)
best = {"distance": float('inf')}

info("Running hybrid search (noisy sampler).")
for idx, group_A in enumerate(itertools.combinations(patient_indices, 3), start=1):
    patient_names = [locations[i] for i in group_A]
    info(f"\n[{idx:02d}/10] Trip A patients: {patient_names}")

    group_B = tuple(p for p in patient_indices if p not in group_A)
    trip_A_nodes = (0,) + group_A
    dist_A_local = distance_matrix[np.ix_(trip_A_nodes, trip_A_nodes)]
    n_local = dist_A_local.shape[0]
    dbg(f"n_local={n_local} -> binary_vars={n_local*(n_local-1)}")

    qp_local, qubo_local = build_tsp_quadratic_program(dist_A_local, penalty=penalty)

    # Instantiate QAOA (try a few signatures quietly)
    qaoa = None
    qaoa_errors = []
    attempts = [
        {"sampler": sampler, "optimizer": optimizer, "reps": 2, "callback": qaoa_callback},
        {"optimizer": optimizer, "sampler": sampler, "reps": 2, "callback": qaoa_callback},
        {"sampler": sampler, "reps": 2, "optimizer": optimizer, "callback": qaoa_callback},
        {"sampler": sampler, "reps": 2, "optimizer": optimizer},
    ]
    for kwargs in attempts:
        try:
            qaoa = QAOA(**kwargs)
            dbg("QAOA created with:", {k:v for k,v in kwargs.items() if k not in ('sampler','callback')})
            break
        except Exception as e:
            qaoa_errors.append((kwargs, e))
            dbg("QAOA attempt failed:", type(e).__name__, e)

    if qaoa is None:
        info("QAOA instantiation failed for this environment. (run with VERBOSE=True for details)")
        if VERBOSE:
            for kw, err in qaoa_errors:
                dbg("Attempt:", {k:(v if k!='sampler' else "<sampler>") for k,v in kw.items()}, "->", repr(err))
        raise RuntimeError("QAOA could not be instantiated.")

    try:
        meo = MinimumEigenOptimizer(min_eigen_solver=qaoa)
    except TypeError:
        meo = MinimumEigenOptimizer(qaoa)

    # Run solver and time it
    t0 = time.time()
    try:
        result = meo.solve(qubo_local)
    except Exception:
        dbg("Exception during solve:")
        traceback.print_exc()
        raise
    t1 = time.time()
    elapsed = t1 - t0

    # concise diagnostics
    fval = getattr(result, "fval", None)
    solver_result = getattr(result, "min_eigen_solver_result", None)
    eigval = None
    if solver_result is not None:
        eigval = getattr(solver_result, "eigenvalue", None)
        # also try alternate common names
        if eigval is None:
            eigval = getattr(solver_result, "min_eigenvalue", None)

    # show short summary: time, fval, eigenvalue (if any), loss history summary
    lh_summary = ""
    if loss_history:
        try:
            lh_summary = f"loss(first/last/min)={loss_history[0]:.6f}/{loss_history[-1]:.6f}/{min(loss_history):.6f}"
        except Exception:
            lh_summary = "loss_history present"
    else:
        lh_summary = "no loss_history"

    info(f"  -> time: {elapsed:.3f}s  raw_obj: {fval if fval is not None else 'N/A'}  eigenval: {eigval if eigval is not None else 'N/A'}  ({lh_summary})")

    # if VERBOSE, show more solver_result internals
    if VERBOSE and solver_result is not None:
        dbg("  solver_result fields (selected):")
        for field in ("optimal_parameters","optimal_point","eigenstate","aux_operators_evaluated"):
            if hasattr(solver_result, field):
                dbg(f"   - {field}: {getattr(solver_result, field)}")

    # Reconstruct path (fallback brute-force if needed)
    path_local = reconstruct_tour_from_solution(result.variables_dict)
    if len(path_local) < n_local:
        dbg("  Warning: extracted path incomplete:", path_local)

    if len(path_local) == n_local and path_local[0] == 0:
        tour = path_local + ([0] if path_local[-1] != 0 else [])
        tripA_dist = sum(float(dist_A_local[a,b]) for a,b in zip(tour[:-1], tour[1:]))
        path_global = [trip_A_nodes[i] for i in path_local]
        if path_global[-1] != 0:
            path_global = path_global + [0]
    else:
        selected = sorted([j for j in range(1,n_local) if any((f"x_{i}_{j}" in result.variables_dict and result.variables_dict[f"x_{i}_{j}"]>0.5) for i in range(n_local) if i!=j)])
        if not selected:
            raise RuntimeError("QAOA returned no selected patient edges; aborting.")
        best_perm = None; best_perm_cost = math.inf
        for perm in itertools.permutations(selected):
            tour = [0] + list(perm) + [0]
            cost = sum(float(dist_A_local[a,b]) for a,b in zip(tour[:-1], tour[1:]))
            if cost < best_perm_cost:
                best_perm_cost = cost; best_perm = perm
        tripA_dist = best_perm_cost
        path_global = [trip_A_nodes[i] for i in ([0] + list(best_perm) + [0])]
        dbg("  Fallback selected subset:", [trip_A_nodes[i] for i in selected], "best_perm:", best_perm)

    # Trip B classical exact (2 patients)
    p1, p2 = group_B
    dist_B_option1 = distance_matrix[0,p1] + distance_matrix[p1,p2] + distance_matrix[p2,0]
    dist_B_option2 = distance_matrix[0,p2] + distance_matrix[p2,p1] + distance_matrix[p1,0]
    if dist_B_option1 <= dist_B_option2:
        tripB_dist = float(dist_B_option1); pathB_global = [0, p1, p2, 0]
    else:
        tripB_dist = float(dist_B_option2); pathB_global = [0, p2, p1, 0]

    total = float(tripA_dist) + float(tripB_dist)
    info(f"  Trip A dist: {tripA_dist:.6f}  Trip B dist: {tripB_dist:.6f}  TOTAL: {total:.6f}")

    if total < best["distance"]:
        best.update({
            "distance": total,
            "trip_A_group": [locations[i] for i in group_A],
            "trip_B_group": [locations[i] for i in group_B],
            "trip_A_route": [locations[i] for i in path_global],
            "trip_B_route": [locations[i] for i in pathB_global],
            "trip_A_dist": tripA_dist,
            "trip_B_dist": tripB_dist,
            "qaoa_time": elapsed,
            "qaoa_sampler_constructor": type(sampler).__name__
        })
        info("  >> New best solution recorded.")

# Final results (compact)
info("\n" + "="*50)
info("FINAL BEST SOLUTION (quantum Trip A, noisy simulator):")
info(f" Total distance (hybrid): {best['distance']:.6f}")
info(f" Trip A (3 patients): {best['trip_A_group']}  Route: {' -> '.join(best['trip_A_route'])}  Distance: {best['trip_A_dist']:.6f}")
info(f" Trip B (2 patients): {best['trip_B_group']}  Route: {' -> '.join(best['trip_B_route'])}  Distance: {best['trip_B_dist']:.6f}")
info(f" Sampler: {best.get('qaoa_sampler_constructor')}  QAOA time (s): {best.get('qaoa_time'):.3f}")
info("="*50)


Setup: FakeProvider GenericBackendV2 -> noise model with 12 qubits (for Trip A up to 3 patients).
Using sampler: SamplerV2
Running hybrid search (noisy sampler).

[01/10] Trip A patients: ['DT', 'GR', 'R2']


/usr/local/lib/python3.12/dist-packages/qiskit_optimization/minimum_eigensolvers/qaoa.py:122: UserWarning: Using Sampler V2 (other than StatevectorSampler) without a pass_manager may result in an error. Consider providing a pass_manager for proper circuit transpilation.
  super().__init__(


  -> time: 3.301s  raw_obj: 47.044999999998254  eigenval: -2147.30632910156  (loss(first/last/min)=-229.939472+0.000000j/-2096.724571+0.000000j/-2147.306329+0.000000j)
  Trip A dist: 47.045000  Trip B dist: 25.740000  TOTAL: 72.785000
  >> New best solution recorded.

[02/10] Trip A patients: ['DT', 'GR', 'R3_2']


/usr/local/lib/python3.12/dist-packages/qiskit_optimization/minimum_eigensolvers/qaoa.py:122: UserWarning: Using Sampler V2 (other than StatevectorSampler) without a pass_manager may result in an error. Consider providing a pass_manager for proper circuit transpilation.
  super().__init__(


  -> time: 2.133s  raw_obj: 35.822000000000116  eigenval: -2137.4271796874987  (loss(first/last/min)=-229.939472+0.000000j/-1982.212479+0.000000j/-2147.306329+0.000000j)
  Trip A dist: 35.822000  Trip B dist: 28.854000  TOTAL: 64.676000
  >> New best solution recorded.

[03/10] Trip A patients: ['DT', 'GR', 'IT']


...

  -> time: 1.948s  raw_obj: 45.60999999999876  eigenval: -2569.248699218751  (loss(first/last/min)=-229.939472+0.000000j/-2447.455988+0.000000j/-3817.554572+0.000000j)
  Trip A dist: 45.610000  Trip B dist: 25.209000  TOTAL: 70.819000

[09/10] Trip A patients: ['GR', 'R3_2', 'IT']


/usr/local/lib/python3.12/dist-packages/qiskit_optimization/minimum_eigensolvers/qaoa.py:122: UserWarning: Using Sampler V2 (other than StatevectorSampler) without a pass_manager may result in an error. Consider providing a pass_manager for proper circuit transpilation.
  super().__init__(


  -> time: 1.782s  raw_obj: 39.11800000000039  eigenval: -3139.6572382812496  (loss(first/last/min)=-229.939472+0.000000j/-3054.148219+0.000000j/-3817.554572+0.000000j)
  Trip A dist: 36.820000  Trip B dist: 31.414000  TOTAL: 68.234000

[10/10] Trip A patients: ['R2', 'R3_2', 'IT']


/usr/local/lib/python3.12/dist-packages/qiskit_optimization/minimum_eigensolvers/qaoa.py:122: UserWarning: Using Sampler V2 (other than StatevectorSampler) without a pass_manager may result in an error. Consider providing a pass_manager for proper circuit transpilation.
  super().__init__(


  -> time: 1.919s  raw_obj: 2022.4680000000008  eigenval: -1640.7348603515622  (loss(first/last/min)=-229.939472+0.000000j/-1302.090650+0.000000j/-3817.554572+0.000000j)
  Trip A dist: 34.332000  Trip B dist: 28.774000  TOTAL: 63.106000

FINAL BEST SOLUTION (quantum Trip A, noisy simulator):
 Total distance (hybrid): 57.523000
 Trip A (3 patients): ['DT', 'GR', 'IT']  Route: H -> DT -> GR -> IT -> H  Distance: 30.736000
 Trip B (2 patients): ['R2', 'R3_2']  Route: H -> R3_2 -> R2 -> H  Distance: 26.787000
 Sampler: SamplerV2  QAOA time (s): 1.792


## Noisy Simulation - Brute force callback for when QAOA returns invalid solution (selects 2 pateints only for example)

In [ ]:

import time, itertools, math, traceback
import numpy as np

VERBOSE = False  # set True for detailed debug output

def info(*a, **k): print(*a, **k)
def dbg(*a, **k):
    if VERBOSE: print(*a, **k)

# Qiskit optimization pieces
from qiskit_optimization import QuadraticProgram
from qiskit_optimization.converters import QuadraticProgramToQubo
from qiskit_optimization.algorithms import MinimumEigenOptimizer
from qiskit_optimization.minimum_eigensolvers import QAOA

# Fake provider + Aer noise
from qiskit.providers.fake_provider import GenericBackendV2
from qiskit_aer import AerSimulator
from qiskit_aer.noise import NoiseModel

# primitive sampler detection
PrimitiveSamplerClass = None
for label, importer in [
    ("qiskit.primitives.Sampler", lambda: __import__("qiskit.primitives", fromlist=["Sampler"]).Sampler),
    ("qiskit_aer.primitives.SamplerV2", lambda: __import__("qiskit_aer.primitives", fromlist=["SamplerV2"]).SamplerV2),
    ("qiskit_aer.primitives.Sampler", lambda: __import__("qiskit_aer.primitives", fromlist=["Sampler"]).Sampler),
]:
    try:
        PrimitiveSamplerClass = importer()
        dbg("Primitive sampler via", label)
        break
    except Exception as e:
        dbg("Sampler import failed for", label, "->", type(e).__name__)

# Optimizer selection / SciPy fallback
optimizer = None
try:
    from qiskit.algorithms.optimizers import SPSA
    optimizer = SPSA(maxiter=50)
    dbg("Using SPSA")
except Exception:
    try:
        from qiskit.algorithms.optimizers import COBYLA
        optimizer = COBYLA(maxiter=200)
        dbg("Using COBYLA")
    except Exception:
        from scipy.optimize import minimize as scipy_minimize
        class SciPyOptimizerWrapper:
            def __init__(self, method='COBYLA', options=None):
                self.method = method
                self.options = options or {"maxiter":200}
            def optimize(self, num_vars, objective, initial_point=None):
                def obj_for_scipy(x):
                    val = objective(x)
                    return float(val) if not isinstance(val, (tuple,list)) else float(val[0])
                x0 = initial_point if initial_point is not None else [0.0]*num_vars
                res = scipy_minimize(obj_for_scipy, x0, method=self.method, options=self.options)
                return res.x, float(res.fun), getattr(res, "nfev", None)
            def minimize(self, fun, x0, **kwargs):
                method = kwargs.pop("method", self.method)
                options = kwargs.pop("options", self.options)
                res = scipy_minimize(fun, x0, method=method, options=options, **kwargs)
                return res
        optimizer = SciPyOptimizerWrapper(method='COBYLA', options={"maxiter":100})
        dbg("Using SciPy optimizer wrapper")

# Problem data
distance_matrix = np.array(
      [[ 0.   ,  8.628, 11.496,  9.445, 10.852,  9.672],
       [14.194,  0.   ,  2.361, 10.922,  9.238,  9.43 ],
       [17.785,  7.745,  0.   , 11.808, 10.124, 10.478],
       [11.864, 19.672, 15.661,  0.   , 11.572, 11.538],
       [ 7.343, 12.172, 10.053,  4.071,  0.   ,  5.931],
       [ 9.269,  9.398, 12.266,  7.318,  8.725,  0.   ]]
)
locations = ['H', 'DT', 'GR', 'R2', 'R3_2', 'IT']
patient_indices = [1,2,3,4,5]
penalty = 1000

# Build fake backend & noise model
max_n_local = 4
expected_qubits = max_n_local * (max_n_local - 1)
info(f"Setup: FakeProvider GenericBackendV2 -> noise model with {expected_qubits} qubits.")
fake_backend = GenericBackendV2(num_qubits=expected_qubits)
try:
    from qiskit_aer import QasmSimulator
    try:
        device_for_noise = QasmSimulator.from_backend(fake_backend)
        noise_model = NoiseModel.from_backend(device_for_noise)
    except Exception:
        noise_model = NoiseModel.from_backend(fake_backend)
except Exception:
    noise_model = NoiseModel.from_backend(fake_backend)
dbg("Noise model built from fake backend.")

# Create AerSimulator defensively
sim_seed = 1234
try:
    simulator = AerSimulator(noise_model=noise_model, method='density_matrix', seed_simulator=sim_seed)
    dbg("AerSimulator created with density_matrix + seed")
except Exception:
    try:
        simulator = AerSimulator(noise_model=noise_model, method='density_matrix')
        dbg("AerSimulator created with density_matrix")
    except Exception:
        simulator = AerSimulator(noise_model=noise_model)
        dbg("AerSimulator created (default)")

# Make primitives sampler if possible
sampler = None
if PrimitiveSamplerClass is not None:
    for ctor in (lambda cls=PrimitiveSamplerClass: cls(backend=simulator),
                 lambda cls=PrimitiveSamplerClass: cls(backend=simulator, shots=1024),
                 lambda cls=PrimitiveSamplerClass: cls()):
        try:
            sampler = ctor()
            break
        except Exception as e:
            dbg("Sampler ctor failed:", type(e).__name__, e)

if sampler is None:
    raise RuntimeError("No primitives Sampler found — install qiskit-aer and ensure qiskit.primitives.Sampler or qiskit_aer.primitives.SamplerV2 is available")

info(f"Using sampler: {type(sampler).__name__}")

# Helper functions
def build_tsp_quadratic_program(dist_local, penalty=penalty):
    n = dist_local.shape[0]
    qp = QuadraticProgram(name="TSP_local")
    for i in range(n):
        for j in range(n):
            if i == j: continue
            qp.binary_var(name=f"x_{i}_{j}")
    linear = {f"x_{i}_{j}": float(dist_local[i,j]) for i in range(n) for j in range(n) if i!=j}
    qp.minimize(linear=linear)
    for i in range(n):
        arrive = {f"x_{j}_{i}": 1 for j in range(n) if j!=i}
        qp.linear_constraint(linear=arrive, sense="==", rhs=1, name=f"arrive_{i}")
        depart = {f"x_{i}_{j}": 1 for j in range(n) if j!=i}
        qp.linear_constraint(linear=depart, sense="==", rhs=1, name=f"depart_{i}")
    qubo = QuadraticProgramToQubo(penalty=penalty).convert(qp)
    return qp, qubo

def reconstruct_tour_from_solution(variables_dict):
    path_dict = {}
    for varname, val in variables_dict.items():
        try:
            if float(val) > 0.5:
                parts = varname.split('_')
                if len(parts) >= 3:
                    si = int(parts[1]); sj = int(parts[2])
                    path_dict[si] = sj
        except Exception:
            continue
    path = [0]; cur = 0
    while True:
        nxt = path_dict.get(cur, None)
        if nxt is None: break
        if nxt in path: break
        path.append(nxt); cur = nxt
        if len(path) > 50: break
    if len(path) >= 1 and path[-1] != 0 and len(path) == len(path_dict)+1:
        path.append(0)
    return path

# QAOA callback & loss history (quiet by default)
loss_history = []
def qaoa_callback(eval_count, parameters, mean, std):
    loss_history.append(mean)
    dbg(f"[QAOA cb] Iter {eval_count}: Energy={mean:.6f} std={std}")

# Main hybrid search
best = {"distance": float('inf')}
info("Running hybrid search (noisy sampler).")

for idx, group_A in enumerate(itertools.combinations(patient_indices, 3), start=1):
    patient_names = [locations[i] for i in group_A]
    info(f"\n[{idx:02d}/10] Trip A patients: {patient_names}")

    group_B = tuple(p for p in patient_indices if p not in group_A)
    trip_A_nodes = (0,) + group_A
    dist_A_local = distance_matrix[np.ix_(trip_A_nodes, trip_A_nodes)]
    n_local = dist_A_local.shape[0]

    qp_local, qubo_local = build_tsp_quadratic_program(dist_A_local, penalty=penalty)

    # instantiate QAOA robustly
    qaoa = None
    qaoa_errors = []
    attempts = [
        {"sampler": sampler, "optimizer": optimizer, "reps": 2, "callback": qaoa_callback},
        {"optimizer": optimizer, "sampler": sampler, "reps": 2, "callback": qaoa_callback},
        {"sampler": sampler, "reps": 2, "optimizer": optimizer, "callback": qaoa_callback},
        {"sampler": sampler, "reps": 2, "optimizer": optimizer},
    ]
    for kwargs in attempts:
        try:
            qaoa = QAOA(**kwargs)
            dbg("QAOA created with", {k:v for k,v in kwargs.items() if k not in ('sampler','callback')})
            break
        except Exception as e:
            qaoa_errors.append((kwargs, e))
            dbg("QAOA attempt failed:", type(e).__name__, e)

    if qaoa is None:
        info("QAOA instantiation failed (set VERBOSE=True for details).")
        raise RuntimeError("QAOA could not be instantiated.")

    try:
        meo = MinimumEigenOptimizer(min_eigen_solver=qaoa)
    except TypeError:
        meo = MinimumEigenOptimizer(qaoa)

    t0 = time.time()
    try:
        result = meo.solve(qubo_local)
    except Exception:
        dbg("Exception during solve:"); traceback.print_exc()
        raise
    t1 = time.time()
    elapsed = t1 - t0

    fval = getattr(result, "fval", None)
    solver_result = getattr(result, "min_eigen_solver_result", None)
    eigval = None
    if solver_result is not None:
        eigval = getattr(solver_result, "eigenvalue", None) or getattr(solver_result, "min_eigenvalue", None)

    if loss_history:
        try:
            lh_summary = f"loss(first/last/min)={loss_history[0]:.6f}/{loss_history[-1]:.6f}/{min(loss_history):.6f}"
        except Exception:
            lh_summary = "loss_history present"
    else:
        lh_summary = "no loss_history"

    info(f"  -> time: {elapsed:.3f}s  raw_obj: {fval if fval is not None else 'N/A'}  eigenval: {eigval if eigval is not None else 'N/A'}  ({lh_summary})")

    # Reconstruct path from QAOA result
    path_local = reconstruct_tour_from_solution(result.variables_dict)
    dbg("Extracted local path:", path_local)

    # Build selected set of patient local indices from result edges (exclude 0)
    selected = sorted([j for j in range(1,n_local) if any((f"x_{i}_{j}" in result.variables_dict and result.variables_dict[f"x_{i}_{j}"]>0.5) for i in range(n_local) if i!=j)])
    dbg("Selected local patient indices by QAOA:", selected)

    # If QAOA found a full Hamiltonian cycle, use it.
    if len(path_local) == n_local and path_local[0] == 0:
        tour = path_local + ([0] if path_local[-1] != 0 else [])
        tripA_dist = sum(float(dist_A_local[a,b]) for a,b in zip(tour[:-1], tour[1:]))
        path_global = [trip_A_nodes[i] for i in path_local]
        if path_global[-1] != 0:
            path_global = path_global + [0]
    else:
        # If QAOA selection is incomplete (not exactly n_local-1 patients), force exact solve among full intended set
        if len(selected) != (n_local - 1):
            dbg("QAOA selected", len(selected), "patients but expected", (n_local - 1))
            info("   > QAOA returned infeasible/partial selection — resolving this subproblem exactly (brute-force) for the full intended patient set.")
            # use full set of intended patients: local indices 1..n_local-1
            selected = list(range(1, n_local))

        # Now brute-force best permutation among 'selected' (which is either QAOA-selected full set or enforced full set)
        best_perm = None; best_perm_cost = math.inf
        for perm in itertools.permutations(selected):
            tour = [0] + list(perm) + [0]
            cost = sum(float(dist_A_local[a,b]) for a,b in zip(tour[:-1], tour[1:]))
            if cost < best_perm_cost:
                best_perm_cost = cost; best_perm = perm
        tripA_dist = best_perm_cost
        path_global = [trip_A_nodes[i] for i in ([0] + list(best_perm) + [0])]
        dbg("Resolved with brute-force best_perm:", best_perm, "cost:", best_perm_cost)

    # Trip B classical exact (2 patients)
    p1, p2 = group_B
    dist_B_option1 = distance_matrix[0,p1] + distance_matrix[p1,p2] + distance_matrix[p2,0]
    dist_B_option2 = distance_matrix[0,p2] + distance_matrix[p2,p1] + distance_matrix[p1,0]
    if dist_B_option1 <= dist_B_option2:
        tripB_dist = float(dist_B_option1); pathB_global = [0, p1, p2, 0]
    else:
        tripB_dist = float(dist_B_option2); pathB_global = [0, p2, p1, 0]

    total = float(tripA_dist) + float(tripB_dist)
    info(f"  Trip A dist: {tripA_dist:.6f}  Trip B dist: {tripB_dist:.6f}  TOTAL: {total:.6f}")

    if total < best["distance"]:
        best.update({
            "distance": total,
            "trip_A_group": [locations[i] for i in group_A],
            "trip_B_group": [locations[i] for i in group_B],
            "trip_A_route": [locations[i] for i in path_global],
            "trip_B_route": [locations[i] for i in pathB_global],
            "trip_A_dist": tripA_dist,
            "trip_B_dist": tripB_dist,
            "qaoa_time": elapsed,
            "qaoa_sampler_constructor": type(sampler).__name__
        })
        info("  >> New best solution recorded.")

# Final results (compact)
info("\n" + "="*50)
info("FINAL BEST SOLUTION (quantum Trip A, noisy simulator):")
info(f" Total distance (hybrid): {best['distance']:.6f}")
info(f" Trip A (3 patients): {best['trip_A_group']}  Route: {' -> '.join(best['trip_A_route'])}  Distance: {best['trip_A_dist']:.6f}")
info(f" Trip B (2 patients): {best['trip_B_group']}  Route: {' -> '.join(best['trip_B_route'])}  Distance: {best['trip_B_dist']:.6f}")
info(f" Sampler: {best.get('qaoa_sampler_constructor')}  QAOA time (s): {best.get('qaoa_time'):.3f}")
info("="*50)


Setup: FakeProvider GenericBackendV2 -> noise model with 12 qubits.
Using sampler: SamplerV2
Running hybrid search (noisy sampler).

[01/10] Trip A patients: ['DT', 'GR', 'R2']


/usr/local/lib/python3.12/dist-packages/qiskit_optimization/minimum_eigensolvers/qaoa.py:122: UserWarning: Using Sampler V2 (other than StatevectorSampler) without a pass_manager may result in an error. Consider providing a pass_manager for proper circuit transpilation.
  super().__init__(


  -> time: 1.818s  raw_obj: 34.66099999999824  eigenval: -2445.070521484373  (loss(first/last/min)=-2363.797479+0.000000j/-2180.865002+0.000000j/-2445.070521+0.000000j)
  Trip A dist: 34.661000  Trip B dist: 25.740000  TOTAL: 60.401000
  >> New best solution recorded.

[02/10] Trip A patients: ['DT', 'GR', 'R3_2']


/usr/local/lib/python3.12/dist-packages/qiskit_optimization/minimum_eigensolvers/qaoa.py:122: UserWarning: Using Sampler V2 (other than StatevectorSampler) without a pass_manager may result in an error. Consider providing a pass_manager for proper circuit transpilation.
  super().__init__(


  -> time: 1.696s  raw_obj: 43.17000000000007  eigenval: -2831.4414521484373  (loss(first/last/min)=-2363.797479+0.000000j/-2705.179550+0.000000j/-2831.441452+0.000000j)
  Trip A dist: 43.170000  Trip B dist: 28.854000  TOTAL: 72.024000

[03/10] Trip A patients: ['DT', 'GR', 'IT']


...

  -> time: 1.704s  raw_obj: 44.052999999999884  eigenval: -2600.076539062498  (loss(first/last/min)=-2363.797479+0.000000j/-2295.895585+0.000000j/-2979.811477+0.000000j)
  Trip A dist: 41.156000  Trip B dist: 25.209000  TOTAL: 66.365000

[09/10] Trip A patients: ['GR', 'R3_2', 'IT']


/usr/local/lib/python3.12/dist-packages/qiskit_optimization/minimum_eigensolvers/qaoa.py:122: UserWarning: Using Sampler V2 (other than StatevectorSampler) without a pass_manager may result in an error. Consider providing a pass_manager for proper circuit transpilation.
  super().__init__(


  -> time: 3.264s  raw_obj: 36.81999999999971  eigenval: -2330.1864902343764  (loss(first/last/min)=-2363.797479+0.000000j/-2287.930192+0.000000j/-2979.811477+0.000000j)
  Trip A dist: 36.820000  Trip B dist: 31.414000  TOTAL: 68.234000

[10/10] Trip A patients: ['R2', 'R3_2', 'IT']


/usr/local/lib/python3.12/dist-packages/qiskit_optimization/minimum_eigensolvers/qaoa.py:122: UserWarning: Using Sampler V2 (other than StatevectorSampler) without a pass_manager may result in an error. Consider providing a pass_manager for proper circuit transpilation.
  super().__init__(


  -> time: 1.782s  raw_obj: 34.332000000000335  eigenval: -2072.2813564453113  (loss(first/last/min)=-2363.797479+0.000000j/-1950.554135+0.000000j/-2979.811477+0.000000j)
  Trip A dist: 34.332000  Trip B dist: 28.774000  TOTAL: 63.106000

FINAL BEST SOLUTION (quantum Trip A, noisy simulator):
 Total distance (hybrid): 57.523000
 Trip A (3 patients): ['DT', 'GR', 'IT']  Route: H -> DT -> GR -> IT -> H  Distance: 30.736000
 Trip B (2 patients): ['R2', 'R3_2']  Route: H -> R3_2 -> R2 -> H  Distance: 26.787000
 Sampler: SamplerV2  QAOA time (s): 2.643
